# Loading datasets and creating folders

In [1]:
!pip install cntk
!pip install -U scikit-learn
!pip install zipfile36
!pip install tqdm

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 5.4MB 307kB/s ta 0:00:01
Requirement not upgraded as not directly required: numpy>=1.8.2 in /data/ontology/anaconda3/envs/conda_env_murali/lib/python3.6/site-packages (from scikit-learn) (1.14.3)
Requirement not upgraded as not directly required: scipy>=0.13.3 in /data/ontology/anaconda3/envs/conda_env_murali/lib/python3.6/site-packages (from scikit-learn) (1.1.0)
  Found existing installation: scikit-learn 0.20.1
    Uninstalling scikit-learn-0.20.1:
      Successfully uninstalled scikit-learn-0.20.1
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip

In [ ]:
!mkdir msaic

In [2]:
%cd msaic
!ls

/data/ontology/Muralidhar/HyperQA/msaic
answer.tsv  eval1_unlabelled.tsv  hyperQA_elmo_100
data.tsv    glove.42B.300d.txt	  hyperQA_elmo_100dummy.txt
data.zip    glove.6B.300d.txt	  hyperQA_elmo_20
elmo2	    glove.840B.300d.txt   hyperQA_elmo_20dummy.txt


In [ ]:
!wget https://competitions.codalab.org/my/datasets/download/2c6a99a5-b071-4f1d-a3b1-d49a923e0c68 
!mv 2c6a99a5-b071-4f1d-a3b1-d49a923e0c68 data.zip
#!mv data.zip msaic/

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
#!mv glove.6B.zip msaic/
glove_zip = r'glove.6B.zip';

#!wget http://nlp.stanford.edu/data/glove.840B.300d.zip
#!mv glove.840B.300d.zip msaic/
#glove_zip = r'glove.840B.300d.zip';

In [ ]:
#!wget https://competitions.codalab.org/my/datasets/download/a6b5bd16-db6f-4cbe-9670-04ecb7504a7a
#!mv a6b5bd16-db6f-4cbe-9670-04ecb7504a7a Starting_Kit.zip
#!mv Starting_Kit.zip msaic/

In [ ]:
import zipfile

zipref = zipfile.ZipFile('data.zip', 'r')
zipref.extractall()
zipref.close()
print('Raw Data Extracted')

zipref = zipfile.ZipFile(glove_zip, 'r')
zipref.extractall()
zipref.close()
print('Glove Data Extracted')

'''
zipref = zipfile.ZipFile('Starting_Kit.zip', 'r')
zipref.extractall()
zipref.close()
print('Starting Kit unzipped')
'''

In [25]:
import nltk
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize, wordpunct_tokenize
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import sys
import re

import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"]='3';
tf_sess_config = tf.ConfigProto()
tf_sess_config.gpu_options.allow_growth = True
'''
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
'''

[nltk_data] Downloading package stopwords to
[nltk_data]     /data/ontology/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /data/ontology/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


"\ndevice_name = tf.test.gpu_device_name()\nif device_name != '/device:GPU:0':\n  raise SystemError('GPU device not found')\nprint('Found GPU at: {}'.format(device_name))\n"

In [4]:
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")
print(cachedStopWords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
# Utility Funcs
def progressBar(value, endvalue, print_vars=[''], print_values=[-1], bar_length=20):
        percent = float(value) / endvalue
        arrow = '-' * int(round(percent * bar_length)-1) + '>'
        spaces = ' ' * (bar_length - len(arrow))
        assert(len(print_vars)==len(print_values));
        strg = '';
        for var, val in zip(print_vars,print_values):
          strg+=' {}: '.format(var.upper());
          strg+='{:5f} |'.format(val);
        sys.stdout.write("\rPercent: [{0}][{1}/{2}] {3}% || {4} ".format(arrow + spaces, int(value), int(endvalue), int(round(percent * 100)), strg))
        sys.stdout.flush()
        return
#progressBar(5,25,['loss','acc'],[5,6])

def cleanText_None(str):
  new_str = [x for x in str if x]
  return new_str, len(new_str)

def cleanText_Simple(str):
  new_str = [x.lower() for x in re.split('\W+', str) if x]
  return new_str, len(new_str)

def cleanText_Spaces_(str_):
  new_str = [x for x in re.split(' ', str_) if x]
  return new_str, len(new_str)
def cleanText_Spaces(str_):
  new_str = [x for x in re.split(' ', str_) if x]
  return ' '.join(new_str), len(new_str)

def cleanText_StopWords(str):
  new_str = [x.lower() for x in re.split('\W+', str) if (x and x not in cachedStopWords)]
  return ' '.join(new_str), len(new_str)


In [6]:
#cleanText = cleanText_Spaces;
cleanText = cleanText_Simple;
print('In the next cell, Change Line 14 and Line 22 accordingly!! ')

In the next cell, Change Line 14 and Line 22 accordingly!! 


In [7]:
# Read data and store it in a dictionary format
data_dict = {};
id2passage = {};
n_queries = -1;
n_passages = -1;
running_query = '';
query_len_counter = np.zeros(1000, dtype='int32');
passage_len_counter = np.zeros(1000, dtype='int32');

f = open('data.tsv','r',encoding='utf-8');
for i, line in enumerate(tqdm(f)):
  tokens = line.strip().split("\t")
  query,passage,label = tokens[1],tokens[2],tokens[3];
  _, cpassage_len = cleanText(passage);
  passage_len_counter[np.min([999,cpassage_len])]+=1;
  n_passages+=1;
  id2passage[n_passages] = passage;
  if running_query!=query:
    running_query = query;
    n_queries+=1;
    data_dict[n_queries] = {};
    _, cquery_len = cleanText(query);
    query_len_counter[np.min([999,cquery_len])]+=1;
    data_dict[n_queries]['query'] = query;
    data_dict[n_queries]['query_len'] = cquery_len;
  if int(label)==1:
    data_dict[n_queries]['pos'] = n_passages;
    data_dict[n_queries]['pos_len'] = cpassage_len;
  else:
    if 'negs' not in data_dict[n_queries]:
      data_dict[n_queries]['negs']=[];
      data_dict[n_queries]['negs_len']=[];
    data_dict[n_queries]['negs'].append(n_passages);
    data_dict[n_queries]['negs_len'].append(cpassage_len);
print('\nTotal No.of Queries: {}, Passages: {}'.format(n_queries+1,n_passages+1));

5241880it [04:03, 21518.33it/s]


Total No.of Queries: 524193, Passages: 5241880


In [8]:
import matplotlib.pyplot as plt
plt.plot(query_len_counter[:50]);
plt.show()
avg_len = np.sum([(i+1)*val for i, val in enumerate(query_len_counter)])/np.sum(query_len_counter);
print('Avg length of queries: {}'.format(avg_len))
plt.plot(passage_len_counter[:500]);
plt.show()
avg_len = np.sum([(i+1)*val for i, val in enumerate(passage_len_counter)])/np.sum(passage_len_counter);
print('Avg length of passages: {}'.format(avg_len))

<Figure size 640x480 with 1 Axes>

Avg length of queries: 6.9925828845482485


<Figure size 640x480 with 1 Axes>

Avg length of passages: 58.581230398254064


In [9]:
# Check if any queries have no right answer!
cnt1 = [];
cnt2 = [];
for id_ in tqdm(data_dict.keys()):
  if 'pos' not in data_dict[id_].keys():
    cnt1.append(id_)
  if 'negs' not in data_dict[id_].keys():
    cnt2.append(id_)
print('');
print(cnt1,cnt2)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 524193/524193 [00:00<00:00, 983493.28it/s]


[51949, 280200, 357523, 434822, 519532] []


In [10]:
# Remove queries that have no right or at least one wrong answers
for id_ in (cnt1+cnt2):
  del data_dict[id_]

In [11]:
# Printing some queries
for q_id in np.random.choice(range(n_queries), size=100, replace=False):  
    print('----------')
    print(data_dict[q_id]['query'])

----------
definition of heirs and assigns
----------
total number of americans killed in all wars
----------
what is the area code for san jose ca
----------
how much does it cost to remodel my house per square foot
----------
what is evolocumab injection?
----------
what is flit
----------
when must a breach be reported to us cert
----------
who has the record for most touchdowns
----------
how long is your state board license good for as a physician?
----------
what masters degree do you need to be a school librarian
----------
how long before sending out wedding thank you cards
----------
what is considered occupational disease
----------
why is important to strain urine for stone
----------
what does it mean when cold hurts a tooth
----------
what county is argyle, tx in?
----------
cost to furnish an apartment
----------
what is copyright of words
----------
what does carnival mean in mardi gras
----------
how long does concrete patio take to set
----------
when did ella baker bo

In [12]:
# Printing some passages
for i in np.random.choice(range(n_passages), size=20, replace=False):
    print('----------')
    print(id2passage[i])

----------
Not all burn is create equally. Related to point number two, not all burn rate spending is bad or good. As the creator of your business the tough decisions of what to spend money on are ultimately up to you decipher. Just remember that not all kinds of burn are created equal.
----------
Chicago Tourism: Best of Chicago. Chicago, Illinois. The windy city is a cornucopia of modern art, fine dining, cutting edge comedy, and die-hard sports fans. Snap a photo of your reflection in the silver Cloud Gate sculpture at Millennium Park before heading to Grant Park to get hit with the refreshing spray of Buckingham Fountain. There are dozens of museums and theater companies in Chicago, so a cultural experience is never hard to find.
----------
The 7 inch WiFi model was released on October 18, 2013 and the 8.9 inch WiFi model was released on November 7, 2013 in the United States. In September 2014, Amazon released the second generation of the Fire HDX 8.9 model that has a faster proces

In [13]:
# Printing some queries and answers
def printData(query_id):
    print('**************')
    print(data_dict[query_id]['query']);
    print(id2passage[data_dict[query_id]['pos']]);
    for i in data_dict[query_id]['negs']:
        print('---------')
        print(id2passage[i])
        
for q_id in np.random.choice(range(n_queries), size=20, replace=False):  
    printData(q_id)

**************
redwood decking cost per linear foot
Routinely staining your deck will help protect your deck but it is not uncommon for pressure treated decking to split, crack, warp, and turn gray in a few years. Cedar and redwood decking prices range from about $1.25 to $2.00 per linear foot for 5/4x6 stock.Redwood is commonly available on the west coast but is scarce in the eastern United States.igher grade materials and long lengths will cost a premium. With over 100 brands to choose from there is a wide range of prices for vinyl and composite decking. The higher end name brands usually cost between $3.00 to $4.00 per linear foot. The big box stores offer economy composite materials for about half the price.
---------
The average cost of basic cedar decking is $4.63 per linear foot. The minimum cost of basic cedar decking is $3.89 per linear foot. The maximum cost of basic cedar decking is $5.39 per linear foot.his includes treated cedar decking. The minimum cost of mid-grade quali

In [14]:
# some global vars
TRAINING_PERCENT = 0.8
VAL_PERCENT = 0.2

In [15]:
# divide train and test
unique_query_ids = list(data_dict.keys())
np.random.shuffle(unique_query_ids) #in-place
n_train = int(np.floor(len(unique_query_ids)*TRAINING_PERCENT))
n_val = int(np.floor(len(unique_query_ids)*VAL_PERCENT))
training_query_ids, validation_query_ids = unique_query_ids[:n_train],unique_query_ids[n_train:n_train+n_val]
print('TRAINING QUERIES: {}, VALIDATION EQURIES: {}'.format(n_train,n_val))

TRAINING QUERIES: 419350, VALIDATION EQURIES: 104837


In [16]:
# some global vars
UNKNOWN_GLOVE = '_unk_';
UNKNOWN_ELMO = '';

In [17]:
# Obtain Glove Embeddings
EMB_DIMS = 300;
glove_file = r'glove.6B.{}d.txt'.format(EMB_DIMS)
#glove_file = r'glove.840B.{}d.txt'.format(EMB_DIMS)
#glove_file = r'glove.42B.{}d.txt'.format(EMB_DIMS)
dfg = pd.read_csv(glove_file, sep=" ", quoting=3, header=None, index_col=0)
dfg.loc[UNKNOWN_GLOVE]=[0.0]*EMB_DIMS;
print(dfg.shape)

# make a hash dict
word2id = {};
id2word = {};
for i, word in tqdm(enumerate(dfg.index.values)):
  word2id[word] = i;
  id2word[i] = word;

48640it [00:00, 486395.83it/s]

(400001, 300)


400001it [00:00, 744847.83it/s]


# Rough Work



In [ ]:
str_ = "When I lived in Michigan, 1 child at a daycare I believe was around $150.00 per week (divide that by 40 hours and that comes out to about $3.75 and hour). Where I live now, 1 child at a daycare is $80.00 per week ~ which is about $2.00 an hour. I guess the cost depends on the location.11-13-2007, 09:24 AM. memoriesbre. Location: Tunkhannock. 934 posts, read. times. Reputation: 322. I am in Tunkhannock PA and charge $4.00 an hour and take care of infants in my home. am obviously getting the deal of the century here...I pay $3.25/hr per/kid for after care. So that's not that much I guess. I pay my friends daughter (she's 15) once in a while to sit for me and give her $7.00 an hour."
cleanText(str_)

In [ ]:
# Rough Work
#print( np.dot(dfg.loc['u.s.a.'],dfg.loc['u.s.']) )
#print( np.dot(dfg.loc['u.s.a.'],dfg.loc['inr']) )
dfg.loc['u.s.a.'].shape

In [ ]:
re.split('\W+', str)

In [ ]:
!pip install textblob
from textblob import TextBlob

In [ ]:
re.sub(r"[^a-zA-Z0-9 ]+", "", 'Hello 2018, the popul@tion is 12,000 per 2000 sq. feet')
TextBlob('Hello Merands William AWhile 2018 15mw the sence logik popultion ortunately is 12000 per ur mei 2000 sq feet').correct()

# Define Utility funcs and directory to save models

In [18]:
# Utility Funcs
def make_glove_batch_data(batch, infer_max_len=True, max_len=0): # batch is list of sentence/paragraph string
  glove_arr, glove_bmasks, n_tokens = [], [], [];
  cstrg_, cstrg_len_ = [], [];
  for row in batch:
    cstrg__, cstrg_len__ = cleanText(row);
    cstrg_.append(cstrg__);
    cstrg_len_.append(cstrg_len__);
  max_len = np.max(cstrg_len_) if infer_max_len else max_len;
  for i, cstrg in enumerate(cstrg_):
    cstrg_glove = [];    
    if cstrg_len_[i]>=max_len:
      cstrg_glove = cstrg[:max_len].copy();
      n_tokens.append(max_len);
    elif cstrg_len_[i]<max_len:
      cstrg_glove = cstrg.copy(); 
      cstrg_glove+=[UNKNOWN_GLOVE]*(max_len-cstrg_len_[i])
      n_tokens.append(cstrg_len_[i]);
    # as per glove requirement
    cstrg_glove = [word if word in word2id.keys() else UNKNOWN_GLOVE for word in cstrg_glove];
    bmask_glove = [0.0 if word==UNKNOWN_GLOVE else 1.0 for word in cstrg_glove];
    cstrg_glove = [word2id[word] for word in cstrg_glove];
    glove_arr.append(cstrg_glove);
    glove_bmasks.append(bmask_glove);
  return np.vstack(glove_arr), np.vstack(glove_bmasks), n_tokens

def make_elmo_batch_data(batch, infer_max_len=True, max_len=0): # batch is list of sentence/paragraph string
  elmo_arr, elmo_bmasks, n_tokens = [], [], [];
  cstrg_, cstrg_len_ = [], [];
  for row in batch:
    cstrg__, cstrg_len__ = cleanText(row);
    cstrg_.append(cstrg__);
    cstrg_len_.append(cstrg_len__);
  max_len = np.max(cstrg_len_) if infer_max_len else max_len;
  for i, cstrg in enumerate(cstrg_):
    cstrg_elmo = [];    
    if cstrg_len_[i]>=max_len:
      cstrg_elmo = cstrg[:max_len].copy();
      n_tokens.append(max_len);
    elif cstrg_len_[i]<max_len:
      cstrg_elmo = cstrg.copy(); 
      cstrg_elmo+=[UNKNOWN_ELMO]*(max_len-cstrg_len_[i])
      n_tokens.append(cstrg_len_[i]);
    # as per elmo requirement
    elmo_arr.append(cstrg_elmo);
    bmask_elmo = [0.0 if word==UNKNOWN_ELMO else 1.0 for word in cstrg_elmo];
    elmo_bmasks.append(bmask_elmo)
  return elmo_arr, elmo_bmasks, n_tokens


def make_glove_and_elmo_batch_data(batch, infer_max_len=True, max_len=0): # batch is list of sentence/paragraph string
  glove_arr, glove_bmasks, elmo_arr, elmo_bmasks, n_tokens = [], [], [], [], [];
  cstrg_, cstrg_len_ = [], [];
  for row in batch:
    cstrg__, cstrg_len__ = cleanText(row);
    cstrg_.append(cstrg__);
    cstrg_len_.append(cstrg_len__);
  max_len = np.max(cstrg_len_) if infer_max_len else max_len;
  for i, cstrg in enumerate(cstrg_):
    cstrg_glove, cstrg_elmo = [], [];    
    if cstrg_len_[i]>=max_len:
      cstrg_glove, cstrg_elmo = cstrg[:max_len].copy(), cstrg[:max_len].copy();
      n_tokens.append(max_len);
    elif cstrg_len_[i]<max_len:
      cstrg_glove, cstrg_elmo = cstrg.copy(), cstrg.copy(); 
      cstrg_glove+=[UNKNOWN_GLOVE]*(max_len-cstrg_len_[i])
      cstrg_elmo+=[UNKNOWN_ELMO]*(max_len-cstrg_len_[i])
      n_tokens.append(cstrg_len_[i]);
    # as per glove requirement
    cstrg_glove = [word if word in word2id.keys() else UNKNOWN_GLOVE for word in cstrg_glove];
    bmask_glove = [0.0 if word==UNKNOWN_GLOVE else 1.0 for word in cstrg_glove];
    cstrg_glove = [word2id[word] for word in cstrg_glove];
    glove_arr.append(cstrg_glove);
    glove_bmasks.append(bmask_glove);
    # as per elmo requirement
    elmo_arr.append(cstrg_elmo);
    bmask_elmo = [0.0 if word==UNKNOWN_ELMO else 1.0 for word in cstrg_elmo];
    elmo_bmasks.append(bmask_elmo);
  return glove_arr, glove_bmasks, elmo_arr, elmo_bmasks, n_tokens
  #return np.vstack(glove_arr), np.vstack(glove_bmasks), np.vstack(elmo_arr), np.vstack(elmo_bmasks), n_tokens

# HyperQA Training from Scrath with Glove

https://arxiv.org/pdf/1707.07847.pdf

In [ ]:
##############################################
# Notes
# Different queries have same query id. Ex: 7977
# Some queries donot have right answers. Ex: 280200
##############################################

In [ ]:
# Utility Funcs
def make_glove_batch_data(batch, infer_max_len=False, max_len=0): # batch is list of sentence/paragraph string
  arr = [];
  b_masks = [];
  cstrg_, cstrg_len_ = [], [];
  for row in batch:
    cstrg__, cstrg_len__ = cleanText(row);
    cstrg_.append(cstrg__);
    cstrg_len_.append(cstrg_len__);
  max_len = np.max(cstrg_len_) if infer_max_len else max_len;
  for i, cstrg in enumerate(cstrg_):
    if cstrg_len_[i]>max_len:
      cstrg = cstrg[:max_len];
    elif cstrg_len_[i]<max_len:
      cstrg+=[UNKNOWN_GLOVE]*(max_len-cstrg_len_[i])
    cstrg = [word if word in word2id.keys() else UNKNOWN_GLOVE for word in cstrg];
    bmask = [0.0 if word==UNKNOWN_GLOVE else 1.0 for word in cstrg];
    cstrg = [word2id[word] for word in cstrg];
    arr.append(cstrg);
    b_masks.append(bmask);
  return np.vstack(arr), np.vstack(b_masks);

In [ ]:
# HyperQA Tensorflow model

class HyperQA(object):
    def __init__(self, vocab_embedding, emb_dim, projection_dim, margin, lr=0.001):
        # parameters
        self.projection_dim = projection_dim;
        self.margin = margin;
        self.emb_dim = emb_dim;
        # placeholder
        self.keep_prob = tf.placeholder(tf.float32, [])
        self.reg_lambda = tf.placeholder(tf.float32, [])
        self.query = tf.placeholder(tf.int32, [None, None])
        self.query_bmask = tf.placeholder(tf.float32, [None, None])
        self.pos_ans = tf.placeholder(tf.int32, [None, None])
        self.pos_ans_bmask = tf.placeholder(tf.float32, [None, None])
        self.neg_ans = tf.placeholder(tf.int32, [None, None])
        self.neg_ans_bmask = tf.placeholder(tf.float32, [None, None])
        # embedding lookup
        self.embedding = tf.Variable(vocab_embedding, trainable=False, dtype=tf.float32, name='embeddings')
        #self.embedding = tf.get_variable('embeddings', shape=[400001,self.emb_dim], initializer=tf.contrib.layers.xavier_initializer(), trainable=True, dtype=tf.float32)
        self.query_emb = tf.nn.embedding_lookup(self.embedding, self.query) # [None,None,emb_dim]
        self.pos_ans_emb = tf.nn.embedding_lookup(self.embedding, self.pos_ans) # [None,None,emb_dim]
        self.neg_ans_emb = tf.nn.embedding_lookup(self.embedding, self.neg_ans) # [None,None,emb_dim]
        # weight vars # a shared variable
        self.w_p = tf.get_variable("w_p", shape=[self.emb_dim,self.projection_dim],
                                   initializer=tf.contrib.layers.xavier_initializer(), trainable=True, dtype=tf.float32)
        self.b_p = tf.get_variable('b_p', shape=[self.projection_dim], initializer=tf.zeros_initializer(), trainable=True, dtype=tf.float32)
        self.w_f = tf.get_variable("w_f", shape=[1], initializer=tf.contrib.layers.xavier_initializer(), trainable=True, dtype=tf.float32)
        self.b_f = tf.get_variable('b_f', shape=[1], initializer=tf.zeros_initializer(), trainable=True, dtype=tf.float32)
        # projections with input # [None,None,emb_dim] and output # [None,None,proj_emb]
        self.query_proj_ = self.project_fn(self.query_emb, self.query_bmask);
        self.pos_ans_proj_ = self.project_fn(self.pos_ans_emb, self.pos_ans_bmask);
        self.neg_ans_proj_ = self.project_fn(self.neg_ans_emb, self.neg_ans_bmask);
        # dropout
        self.query_proj = tf.nn.dropout(self.query_proj_, self.keep_prob);
        self.pos_ans_proj = tf.nn.dropout(self.pos_ans_proj_, self.keep_prob);
        self.neg_ans_proj = tf.nn.dropout(self.neg_ans_proj_, self.keep_prob);
        # unit normalized representations with output #[None,proj_emb]
        self.query_vec = tf.clip_by_norm(tf.reduce_sum(self.query_proj,axis=1), 1.0, axes=1) 
        self.pos_ans_vec = tf.clip_by_norm(tf.reduce_sum(self.pos_ans_proj,axis=1), 1.0, axes=1)
        self.neg_ans_vec = tf.clip_by_norm(tf.reduce_sum(self.neg_ans_proj,axis=1), 1.0, axes=1)
        # hyperbolic distance
        self.p_distance = self.hyperbolic_ball(self.query_vec, self.pos_ans_vec) #[None,1]
        self.n_distance = self.hyperbolic_ball(self.query_vec, self.neg_ans_vec) #[None,1]
        # loss
        self.p_score = self.p_distance*self.w_f+self.b_f; 
        self.n_score = self.n_distance*self.w_f+self.b_f;
        self.losses = tf.nn.relu(self.margin + self.n_score - self.p_score) #[None,1]
        self.reg_losses = self.reg_lambda*tf.reduce_sum(tf.abs(self.w_p));
        self.loss = self.reg_losses+tf.reduce_sum(self.losses) #[]
        # print loss ops
        self.print_p_distance = tf.reduce_mean(self.p_distance)
        self.print_n_distance = tf.reduce_mean(self.n_distance)
        self.print_p_score_loss = tf.reduce_mean(self.p_score)
        self.print_n_score_loss = tf.reduce_mean(self.n_score)
        self.print_losses = tf.reduce_mean(self.losses)
        # optimizer
        self.optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        # adjust gradient
        gradients = self.optimizer.compute_gradients(self.loss)
        reim_gradients = [(self._to_riemannian_gradient(grad), var) for grad, var in gradients]
        clip_gradients = [(self._ClipIfNotNone(grad), var) for grad, var in reim_gradients]
        self.train_op = self.optimizer.apply_gradients(clip_gradients)       
    def project_fn(self, inp_emb, bmask): # Input Shape [None,None,emb_dim], [None,None]
        runtime_shape = tf.shape(inp_emb);
        dim1 = runtime_shape[0];
        dim2 = runtime_shape[1];
        dense_output = tf.nn.xw_plus_b(tf.reshape(inp_emb, [-1,self.emb_dim]), self.w_p, self.b_p);
        activated_output = tf.nn.relu(dense_output);
        proj_emb = tf.reshape(activated_output,[dim1,dim2,self.projection_dim]);
        bmask = tf.tile(tf.expand_dims(bmask,axis=-1),[1,1,self.projection_dim]);
        masked_proj_emb = bmask*proj_emb;
        return masked_proj_emb
    def hyperbolic_ball(self, x, y, neg=False, eps=1E-6):
        """ Poincare Distance Function """
        z = x - y
        z = tf.norm(z, ord='euclidean', keep_dims=True, axis=1)
        z = tf.square(z)
        x_d = 1 - tf.square(tf.norm(x, ord='euclidean', keep_dims=True, axis=1))
        y_d = 1 - tf.square(tf.norm(y, ord='euclidean', keep_dims=True, axis=1))
        d = x_d * y_d
        z = z / (d + eps)
        z  = (2 * z) + 1
        arcosh = z + tf.sqrt(tf.square(z) - 1 + eps)
        arcosh = tf.log(arcosh)
        if(neg):
            arcosh = -arcosh
        return arcosh
    def _ClipIfNotNone(self, grad):
        if grad is None:
          return grad
        grad = tf.clip_by_value(grad, -10, 10, name=None)
        #grad = tf.clip_by_norm(grad, 1.0)
        return grad
    def _to_riemannian_gradient(self, ge):
      if ge is None:
        return None
      try:
        shape = ge.get_shape().as_list()
        if len(shape) >= 3:
            grad_scale = 1 - tf.square(tf.norm(ge, axis=[-2, -1], keepdims=True))
        elif len(shape) == 2:
            grad_scale = 1 - tf.square(tf.norm(ge, keepdims=True))
        else:
            return ge
      except:
        print('Exception handled!')
        grad_scale = 1 - tf.square(tf.norm(ge, keep_dims=True))
      grad_scale = (tf.square(grad_scale) + 1e-10) / 4.0
      gr = ge * grad_scale
      # gr = tf.clip_by_norm(gr, 1.0, axes=0)
      return gr

In [ ]:
# some global vars
PROJ_DIMS = 200;

# args
MARGIN = 5.0;
LEARNING_RATE = 0.001;
TRAINING_BATCH_SIZE = 64;
VALIDATION_BATCH_SIZE = 1024;
VALIDATION_EVERY = 1;
N_EPOCHS = 100;

# Create a graph with given params
tf.reset_default_graph();
hyperQA_Graph = tf.Graph();
with hyperQA_Graph.as_default():
  hyperQA = HyperQA(dfg.as_matrix(), EMB_DIMS, PROJ_DIMS, MARGIN, LEARNING_RATE)
  print('graph built')

In [ ]:
# NEW: max sampling from https://cs.uwaterloo.ca/~jimmylin/publications/Rao_etal_CIKM2016.pdf
# Run this cell for both random_ as well as max_sampling
choice_max = 0.5;
default_sampling_choice = 'RANDOM';
RESUME_TRAINING = False;
START_EPOCH = 0;

# Run the connections for desired epochs
with tf.Session(graph=hyperQA_Graph, config=tf_sess_config) as sess:
  saver = tf.train.Saver();
  if not RESUME_TRAINING:
    sess.run(tf.global_variables_initializer());
    START_EPOCH = 0;
  else:
    saver.restore(sess, './'+model_save_folder+'/epoch_model_{}.ckpt'.format(START_EPOCH-1));
  for epoch in np.arange(START_EPOCH, N_EPOCHS):
    # sampling choice
    if epoch==0:
      sampling_choice = default_sampling_choice;
    else:
      sampling_choice = 'MAX' if np.random.random()<choice_max else default_sampling_choice;
    # Loss and optimization on Training Data
    print('beginning epoch: {}'.format(epoch));
    n_batches = int(np.floor(n_train/TRAINING_BATCH_SIZE));
    cum_loss = 0;
    cum_pos_loss = 0;
    cum_neg_loss = 0;
    frm=0;
    # TRAIN
    for i in range(n_batches):
      batch_query_ids = training_query_ids[frm:frm+TRAINING_BATCH_SIZE];
      batch_queries = [];
      batch_pos = [];
      batch_neg = [];
      batch_arg_neg = [];
      # TRAIN: Find the most challenging negative!
      if sampling_choice=='MAX': #get j's which are maximums in their negative sets
        dummy_batch_queries = [];
        dummy_batch_neg = [];
        dummy_batch_neg_n = [];
        for query_id in batch_query_ids:
          dummy_batch_queries.append(data_dict[query_id]['query']);
          for jj in data_dict[query_id]['negs']:
            dummy_batch_neg.append(id2passage[jj]);
          dummy_batch_neg_n.append(len(data_dict[query_id]['negs']))
        d_x, d_x1 = make_glove_batch_data(dummy_batch_queries,infer_max_len=True)
        d_z, d_z1 = make_glove_batch_data(dummy_batch_neg,infer_max_len=True)
        d_x_new, d_x1_new = [], [];
        for ii,n_times in enumerate(dummy_batch_neg_n): # to tile x and y for thses many times 
          d_x_new.append(np.tile(d_x[ii],(n_times,1)));
          d_x1_new.append(np.tile(d_x1[ii],(n_times,1)));
        d_x_new, d_x1_new = np.vstack(d_x_new), np.vstack(d_x1_new);
        result = sess.run([hyperQA.n_score],feed_dict={hyperQA.keep_prob: 1.0,
                                                       hyperQA.query:d_x_new,
                                                       hyperQA.query_bmask:d_x1_new,
                                                       hyperQA.neg_ans:d_z,
                                                       hyperQA.neg_ans_bmask:d_z1})
        result = result[0]; # array of batch size i.e. (BS,1)
        k=0;
        for n_times in dummy_batch_neg_n: # because no of neg answers per query is variable
          batch_arg_neg.append(np.argmax(result[k:k+n_times]));
          k+=n_times;
      # TRAIN: compute loss and optimize
      if sampling_choice=='MAX':
        assert(len(batch_arg_neg)==len(batch_query_ids))
      for i_id, query_id in enumerate(batch_query_ids):
        batch_queries.append(data_dict[query_id]['query']);
        batch_pos.append(id2passage[data_dict[query_id]['pos']]);
        this_negative = id2passage[data_dict[query_id]['negs'][batch_arg_neg[i_id]]] if sampling_choice=='MAX' else id2passage[np.random.choice(data_dict[query_id]['negs'])];
        batch_neg.append(this_negative);
      x, x1 = make_glove_batch_data(batch_queries,infer_max_len=True)
      y, y1 = make_glove_batch_data(batch_pos,infer_max_len=True) 
      z, z1 = make_glove_batch_data(batch_neg,infer_max_len=True)
      result = sess.run([hyperQA.train_op, hyperQA.print_losses, hyperQA.print_p_score_loss, hyperQA.print_n_score_loss],
                        feed_dict={hyperQA.keep_prob: 0.8,
                                   hyperQA.reg_lambda: 0.00001,
                                   hyperQA.query:x,
                                   hyperQA.query_bmask:x1,
                                   hyperQA.pos_ans:y,
                                   hyperQA.pos_ans_bmask:y1,
                                   hyperQA.neg_ans:z,
                                   hyperQA.neg_ans_bmask:z1})
      cum_loss+=result[1];
      cum_pos_loss+=result[2];
      cum_neg_loss+=result[3];
      frm+=TRAINING_BATCH_SIZE;
      progressBar(frm,n_train,['loss','pos_loss','neg_loss'],[result[1],result[2],result[3]]);
    print('\nTRAINING || mean loss: {} | mean pos loss: {} | mean neg loss: {}'.format(cum_loss/n_batches,cum_pos_loss/n_batches,cum_neg_loss/n_batches))
    saver.save(sess, './'+model_save_folder+'/epoch_model_{}.ckpt'.format(epoch));
    # VALIDATE
    if epoch%VALIDATION_EVERY==0:
      #saver = tf.train.Saver();
      #saver.restore(val_sess, './'+model_save_folder+'/epoch_model_20.ckpt');
      n_batches = int(np.floor(n_val/VALIDATION_BATCH_SIZE));
      cum_loss = 0;
      cum_pos_loss = 0;
      cum_neg_loss = 0;
      frm=0;
      for i in range(n_batches):
        batch_query_ids = validation_query_ids[frm:frm+VALIDATION_BATCH_SIZE];
        batch_queries = [];
        batch_pos = [];
        batch_neg = [];
        batch_arg_neg = [];
        # VALIDATE: Find the most challenging negative!
        if sampling_choice=='MAX': #get j's which are maximums in their negative sets
          dummy_batch_queries = [];
          dummy_batch_neg = [];
          dummy_batch_neg_n = [];
          for query_id in batch_query_ids:
            dummy_batch_queries.append(data_dict[query_id]['query']);
            for jj in data_dict[query_id]['negs']:
              dummy_batch_neg.append(id2passage[jj]);
            dummy_batch_neg_n.append(len(data_dict[query_id]['negs']))
          d_x, d_x1 = make_glove_batch_data(dummy_batch_queries,infer_max_len=True)
          d_z, d_z1 = make_glove_batch_data(dummy_batch_neg,infer_max_len=True)
          d_x_new, d_x1_new = [], [];
          for ii,n_times in enumerate(dummy_batch_neg_n): # to tile x and y for thses many times 
            d_x_new.append(np.tile(d_x[ii],(n_times,1)));
            d_x1_new.append(np.tile(d_x1[ii],(n_times,1)));
          d_x_new, d_x1_new = np.vstack(d_x_new), np.vstack(d_x1_new);
          result = sess.run([hyperQA.n_score],feed_dict={hyperQA.keep_prob: 1.0,
                                                         hyperQA.query:d_x_new,
                                                         hyperQA.query_bmask:d_x1_new,
                                                         hyperQA.neg_ans:d_z,
                                                         hyperQA.neg_ans_bmask:d_z1})
          result = result[0]; # array of batch size i.e. (BS,1)
          k=0;
          for n_times in dummy_batch_neg_n: # because no of neg answers per query is variable
            batch_arg_neg.append(np.argmax(result[k:k+n_times]));
            k+=n_times;
        # VALIDATE: compute loss only
        if sampling_choice=='MAX':
          assert(len(batch_arg_neg)==len(batch_query_ids))
        for i_id, query_id in enumerate(batch_query_ids):
          batch_queries.append(data_dict[query_id]['query']);
          batch_pos.append(id2passage[data_dict[query_id]['pos']]);
          this_negative = id2passage[data_dict[query_id]['negs'][batch_arg_neg[i_id]]] if sampling_choice=='MAX' else id2passage[np.random.choice(data_dict[query_id]['negs'])];
          batch_neg.append(this_negative);
        x, x1 = make_glove_batch_data(batch_queries,infer_max_len=True)
        y, y1 = make_glove_batch_data(batch_pos,infer_max_len=True) 
        z, z1 = make_glove_batch_data(batch_neg,infer_max_len=True)
        result = sess.run([hyperQA.print_losses, hyperQA.print_p_score_loss, hyperQA.print_n_score_loss],
                          feed_dict={hyperQA.keep_prob:1.0,
                                     hyperQA.reg_lambda:0.0,
                                     hyperQA.query:x,
                                     hyperQA.query_bmask:x1,
                                     hyperQA.pos_ans:y,
                                     hyperQA.pos_ans_bmask:y1,
                                     hyperQA.neg_ans:z,
                                     hyperQA.neg_ans_bmask:z1})
        cum_loss+=result[0];
        cum_pos_loss+=result[1];
        cum_neg_loss+=result[2];
        frm+=VALIDATION_BATCH_SIZE;
        progressBar(frm,n_val,['loss','pos_loss','neg_loss'],[result[0],result[1],result[2]]);
      print('\nVALIDATION || mean loss: {} | mean pos loss: {} | mean neg loss: {}'.format(cum_loss/n_batches,cum_pos_loss/n_batches,cum_neg_loss/n_batches))

In [ ]:
# args
EVAL_BATCH_SIZE = 1024;

# load data
df_test = pd.read_csv("eval1_unlabelled.tsv", sep= '\t', header=None)
print('Eval Data Loaded.')

In [ ]:
# get scores
outputs = [];
with tf.Session(graph=hyperQA_Graph, config=tf_sess_config) as sess:
  saver = tf.train.Saver();
  saver.restore(sess, r'./'+model_save_folder+'/epoch_model_20.ckpt');
  for i, row in df_test.iterrows():
    if i==0:
      batch_queries = [];
      batch_passages = [];
      batch_ql_max = -1;
      batch_pl_max = -1;
    if i!=0 and i%EVAL_BATCH_SIZE==0:
      x, x1 = make_glove_batch_data(batch_queries,infer_max_len=True);
      y, y1 = make_glove_batch_data(batch_passages,infer_max_len=True);
      result = sess.run([hyperQA.p_score],feed_dict={hyperQA.keep_prob: 1.0,
                                                     hyperQA.query:x,
                                                     hyperQA.query_bmask:x1,
                                                     hyperQA.pos_ans:y,
                                                     hyperQA.pos_ans_bmask:y1})
      outputs.append(result[0].tolist());
      progressBar(i,df_test.shape[0]);
      batch_queries = [];
      batch_passages = [];
      batch_ql_max = -1;
      batch_pl_max = -1;
    query, passage = row[1], row[2];
    batch_queries.append(query);
    batch_passages.append(passage);
  if batch_queries:
    x, x1 = make_glove_batch_data(batch_queries,infer_max_len=True);
    y, y1 = make_glove_batch_data(batch_passages,infer_max_len=True);
    result = sess.run([hyperQA.p_score],feed_dict={hyperQA.keep_prob: 1.0,
                                                   hyperQA.query:x,
                                                   hyperQA.query_bmask:x1,
                                                   hyperQA.pos_ans:y,
                                                   hyperQA.pos_ans_bmask:y1})
    outputs.append(result[0].tolist());
    progressBar(i,df_test.shape[0]);
    batch_queries = [];
    batch_passages = [];
    batch_ql_max = -1;
    batch_pl_max = -1;

In [ ]:
outputs = np.vstack(outputs)
df_test['pred'] = outputs; # the smaller the score, the more positive_answer it is

In [ ]:
# re-using code from Dileep
outfilename = 'answer.tsv'
with open(outfilename,"w",encoding="utf-8") as fw:
  import math
  linelist = []
  tempscores = []
  for idx, row in tqdm(df_test.iterrows()):
      tempscores.append(row['pred'])
      #tempscores.append(str(row['pred']))
      if((idx +1)%10==0):
          tempscores-=np.min(tempscores);
          tempscores = [math.exp(s) for s in tempscores];
          expsum = sum(tempscores)
          tempscores = [str(s/expsum) for s in tempscores]
          scoreString = "\t".join(tempscores)
          qid = str(row[0])
          fw.write(qid+"\t"+scoreString+"\n")
          tempscores=[]
      #if(idx%10000==0):
      #    print(idx)

# HyperQA Training from Scrath with Glove and ELMo

In [19]:
# model save 
!mkdir hyperQA_elmo_100
model_save_folder = r'hyperQA_elmo_100'
dfile = open('./'+model_save_folder+'/dummy.txt','w',encoding='utf-8');
dfile.write('Checking...');
dfile.close();

In [ ]:
# https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/semantic_similarity_with_tf_hub_universal_encoder.ipynb
# https://tfhub.dev/google/elmo/2
# https://tfhub.dev/google/universal-sentence-encoder/2
# https://github.com/PrashantRanjan09/WordEmbeddings-Elmo-Fasttext-Word2Vec

In [20]:
import tensorflow_hub as hub

In [21]:
# HyperQA Tensorflow model

class HyperQA_ELMo(object):
    def __init__(self, vocab_embedding, glove_emb_dim, projection_dim, margin, lr=0.001, elmo=None, elmo_emb_dim=1024):

        # parameters and variables
        self.glove_embedding = tf.Variable(vocab_embedding, trainable=False, dtype=tf.float32, name='glove_embedding')
        self.glove_emb_dim = glove_emb_dim;
        self.projection_dim = projection_dim;
        self.margin = margin;
        self.lr = lr;
        self.elmo = elmo;
        self.elmo_emb_dim = elmo_emb_dim;
        self.concat_dim = self.glove_emb_dim+self.elmo_emb_dim;
        self.init_name = tf.contrib.layers.xavier_initializer()
        self.w_p = tf.get_variable("w_p", shape=[self.concat_dim,self.projection_dim], initializer=self.init_name, trainable=True, dtype=tf.float32)
        self.b_p = tf.get_variable('b_p', shape=[self.projection_dim], initializer=tf.zeros_initializer(), trainable=True, dtype=tf.float32)
        self.w_f = tf.get_variable("w_f", shape=[1], initializer=self.init_name, trainable=True, dtype=tf.float32)
        self.b_f = tf.get_variable('b_f', shape=[1], initializer=tf.zeros_initializer(), trainable=True, dtype=tf.float32)

        # placeholder: general
        self.keep_prob = tf.placeholder(tf.float32, [])
        self.reg_lambda = tf.placeholder(tf.float32, [])
        # placeholder: glove
        self.query_glove = tf.placeholder(tf.int32, [None, None])
        self.pos_ans_glove = tf.placeholder(tf.int32, [None, None])
        self.neg_ans_glove = tf.placeholder(tf.int32, [None, None])
        # placeholder: elmo
        self.query_elmo = tf.placeholder(tf.string, shape=(None, None));
        self.query_elmo_tokens = tf.placeholder(tf.int32, shape=(None));
        self.pos_ans_elmo = tf.placeholder(tf.string, shape=(None, None));
        self.pos_ans_elmo_tokens = tf.placeholder(tf.int32, shape=(None));
        self.neg_ans_elmo = tf.placeholder(tf.string, shape=(None, None));
        self.neg_ans_elmo_tokens = tf.placeholder(tf.int32, shape=(None));
        # placeholder: masking
        self.query_bmask = tf.placeholder(tf.float32, [None, None])
        self.pos_ans_bmask = tf.placeholder(tf.float32, [None, None])
        self.neg_ans_bmask = tf.placeholder(tf.float32, [None, None])


        # get glove embedding # [None,None,glove_emb_dim]
        self.query_glove_emb = tf.nn.embedding_lookup(self.glove_embedding, self.query_glove)
        self.pos_ans_glove_emb = tf.nn.embedding_lookup(self.glove_embedding, self.pos_ans_glove)
        self.neg_ans_glove_emb = tf.nn.embedding_lookup(self.glove_embedding, self.neg_ans_glove) 

        # get elmo embeddings # [None,None,elmo_emb_dim]
        self.query_elmo_emb = self.ElmoEmbedding(self.query_elmo, self.query_elmo_tokens);
        self.pos_ans_elmo_emb = self.ElmoEmbedding(self.pos_ans_elmo, self.pos_ans_elmo_tokens);
        self.neg_ans_elmo_emb = self.ElmoEmbedding(self.neg_ans_elmo, self.neg_ans_elmo_tokens);

        # concat embeddings # [None,None,concat_dim]
        self.query_emb = tf.concat([self.query_glove_emb,self.query_elmo_emb],axis=-1)
        self.pos_ans_emb = tf.concat([self.pos_ans_glove_emb,self.pos_ans_elmo_emb],axis=-1)
        self.neg_ans_emb = tf.concat([self.neg_ans_glove_emb,self.neg_ans_elmo_emb],axis=-1)

        # projections with input # [None,None,glove_emb_dim] and output # [None,None,proj_emb]
        self.query_proj_ = self.project_fn(self.query_emb, self.query_bmask);
        self.pos_ans_proj_ = self.project_fn(self.pos_ans_emb, self.pos_ans_bmask);
        self.neg_ans_proj_ = self.project_fn(self.neg_ans_emb, self.neg_ans_bmask);
        # dropout
        self.query_proj = tf.nn.dropout(self.query_proj_, self.keep_prob);
        self.pos_ans_proj = tf.nn.dropout(self.pos_ans_proj_, self.keep_prob);
        self.neg_ans_proj = tf.nn.dropout(self.neg_ans_proj_, self.keep_prob);
        # unit normalized representations with output #[None,proj_emb]
        self.query_vec = tf.clip_by_norm(tf.reduce_sum(self.query_proj,axis=1), 1.0, axes=1) 
        self.pos_ans_vec = tf.clip_by_norm(tf.reduce_sum(self.pos_ans_proj,axis=1), 1.0, axes=1)
        self.neg_ans_vec = tf.clip_by_norm(tf.reduce_sum(self.neg_ans_proj,axis=1), 1.0, axes=1)
        # hyperbolic distance
        self.p_distance = self.hyperbolic_ball(self.query_vec, self.pos_ans_vec) #[None,1]
        self.n_distance = self.hyperbolic_ball(self.query_vec, self.neg_ans_vec) #[None,1]
        # loss
        self.p_score = self.p_distance*self.w_f+self.b_f; 
        self.n_score = self.n_distance*self.w_f+self.b_f;
        self.losses = tf.nn.relu(self.margin + self.n_score - self.p_score) #[None,1]
        self.reg_losses = self.reg_lambda*tf.reduce_sum(tf.abs(self.w_p));
        self.loss = self.reg_losses+tf.reduce_sum(self.losses) #[]
        # print loss ops
        self.print_p_distance = tf.reduce_mean(self.p_distance)
        self.print_n_distance = tf.reduce_mean(self.n_distance)
        self.print_p_score_loss = tf.reduce_mean(self.p_score)
        self.print_n_score_loss = tf.reduce_mean(self.n_score)
        self.print_losses = tf.reduce_mean(self.losses)
        # optimizer
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.lr)
        # adjust gradient
        gradients = self.optimizer.compute_gradients(self.loss)
        reim_gradients = [(self._to_riemannian_gradient(grad), var) for grad, var in gradients]
        clip_gradients = [(self._ClipIfNotNone(grad), var) for grad, var in reim_gradients]
        self.train_op = self.optimizer.apply_gradients(clip_gradients)  

    def ElmoEmbedding(self, tokens_input, tokens_length): #inputs here are tensors 
        return self.elmo(inputs={"tokens": tokens_input,"sequence_len": tokens_length},signature="tokens",as_dict=True)["elmo"]     
    def project_fn(self, inp_emb, bmask): # Input Shape [None,None,concat_dim], [None,None]
        runtime_shape = tf.shape(inp_emb);
        dim1 = runtime_shape[0];
        dim2 = runtime_shape[1];
        dense_output = tf.nn.xw_plus_b(tf.reshape(inp_emb, [-1,self.concat_dim]), self.w_p, self.b_p);
        activated_output = tf.nn.relu(dense_output);
        proj_emb = tf.reshape(activated_output,[dim1,dim2,self.projection_dim]);
        bmask = tf.tile(tf.expand_dims(bmask,axis=-1),[1,1,self.projection_dim]);
        masked_proj_emb = bmask*proj_emb;
        return masked_proj_emb
    def hyperbolic_ball(self, x, y, neg=False, eps=1E-6):
        """ Poincare Distance Function """
        z = x - y
        z = tf.norm(z, ord='euclidean', keep_dims=True, axis=1)
        z = tf.square(z)
        x_d = 1 - tf.square(tf.norm(x, ord='euclidean', keep_dims=True, axis=1))
        y_d = 1 - tf.square(tf.norm(y, ord='euclidean', keep_dims=True, axis=1))
        d = x_d * y_d
        z = z / (d + eps)
        z  = (2 * z) + 1
        arcosh = z + tf.sqrt(tf.square(z) - 1 + eps)
        arcosh = tf.log(arcosh)
        if(neg):
            arcosh = -arcosh
        return arcosh
    def _ClipIfNotNone(self, grad):
        if grad is None:
          return grad
        grad = tf.clip_by_value(grad, -10, 10, name=None)
        #grad = tf.clip_by_norm(grad, 1.0)
        return grad
    def _to_riemannian_gradient(self, ge):
      if ge is None:
        return None
      try:
        shape = ge.get_shape().as_list()
        if len(shape) >= 3:
            grad_scale = 1 - tf.square(tf.norm(ge, axis=[-2, -1], keepdims=True))
        elif len(shape) == 2:
            grad_scale = 1 - tf.square(tf.norm(ge, keepdims=True))
        else:
            return ge
      except:
        print('Exception handled!')
        grad_scale = 1 - tf.square(tf.norm(ge, keep_dims=True))
      grad_scale = (tf.square(grad_scale) + 1e-10) / 4.0
      gr = ge * grad_scale
      # gr = tf.clip_by_norm(gr, 1.0, axes=0)
      return gr

In [22]:
# some global vars
PROJ_DIMS = 100;
MARGIN = 5.0;
LEARNING_RATE = 0.001;
TRAINING_BATCH_SIZE = 32;
VALIDATION_BATCH_SIZE = 32;
VALIDATION_EVERY = 1;
N_EPOCHS = 100;

# Create a graph with given params
tf.reset_default_graph();
elmo = hub.Module("elmo2", trainable=True)
hyperQA_ELMo = HyperQA_ELMo(dfg.values, EMB_DIMS, PROJ_DIMS, MARGIN, LEARNING_RATE, elmo=elmo, elmo_emb_dim=1024)
print('graph built')

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
Instructions for updating:
keep_dims is deprecated, use keepdims instead
graph built


In [ ]:
# NEW: max sampling from https://cs.uwaterloo.ca/~jimmylin/publications/Rao_etal_CIKM2016.pdf
# Run this cell for both random_ as well as max_sampling
choice_max = 0; # the greater this is, the more cprob to chosse MAX over RANDOM
default_sampling_choice = 'RANDOM';
RESUME_TRAINING = True;
START_EPOCH = 1;

# Run the connections for desired epochs
with tf.Session(config=tf_sess_config) as sess:
  saver = tf.train.Saver();
  if not RESUME_TRAINING:
    sess.run(tf.global_variables_initializer());
    START_EPOCH = 0;
  else:
    saver.restore(sess, './'+model_save_folder+'/epoch_model_{}.ckpt'.format(START_EPOCH-1));
  for epoch in np.arange(START_EPOCH, N_EPOCHS):
    # sampling choice
    if epoch==0:
      sampling_choice = default_sampling_choice;
    else:
      sampling_choice = 'MAX' if np.random.random()<choice_max else default_sampling_choice;
    # Loss and optimization on Training Data
    print('beginning epoch: {}'.format(epoch));
    n_batches = int(np.floor(n_train/TRAINING_BATCH_SIZE));
    cum_loss = 0;
    cum_pos_loss = 0;
    cum_neg_loss = 0;
    frm=0;
    # TRAIN
    for i in range(n_batches):
      if i%1000==0:
        saver.save(sess, './'+model_save_folder+'/batch_model.ckpt'.format(epoch));
      batch_query_ids = training_query_ids[frm:frm+TRAINING_BATCH_SIZE];
      batch_queries = [];
      batch_pos = [];
      batch_neg = [];
      batch_arg_neg = [];
      # TRAIN: Find the most challenging negative!
      if sampling_choice=='MAX': #get j's which are maximums in their negative sets
        unfort_index = 0;
        while(unfort_index<len(batch_query_ids)):
            small_batch_query_ids = batch_query_ids[unfort_index:np.min([unfort_index+3,len(batch_query_ids)])];
            dummy_batch_queries = [];
            dummy_batch_neg = [];
            dummy_batch_neg_n = [];
            for query_id in small_batch_query_ids:
              dummy_batch_queries.append(data_dict[query_id]['query']);
              for jj in data_dict[query_id]['negs']:
                dummy_batch_neg.append(id2passage[jj]);
              dummy_batch_neg_n.append(len(data_dict[query_id]['negs']))        
            #glove_arr, glove_bmasks, elmo_arr, elmo_bmasks, n_tokens = make_glove_and_elmo_batch_data(test_batch)
            aa1, _, aa3, aa4, aa5 = make_glove_and_elmo_batch_data(dummy_batch_queries,infer_max_len=True)
            bb1, _, bb3, bb4, bb5 = make_glove_and_elmo_batch_data(dummy_batch_neg,infer_max_len=True)
            aa1_new, aa3_new, aa4_new, aa5_new = [], [], [], [];
            for ii,n_times in enumerate(dummy_batch_neg_n): # to tile x and y for thses many times 
              aa1_new.append(np.tile(aa1[ii],(n_times,1)));
              aa3_new.append(np.tile(aa3[ii],(n_times,1)));
              aa4_new.append(np.tile(aa4[ii],(n_times,1)));
              aa5_new.append(np.tile(aa5[ii],(n_times,1)));
            aa1_new, aa3_new, aa4_new, aa5_new = np.vstack(aa1_new), np.vstack(aa3_new), np.vstack(aa4_new), np.vstack(aa5_new);
            aa5_new = np.squeeze(aa5_new) # making it 1-dim
            result = sess.run([hyperQA_ELMo.n_score],feed_dict={hyperQA_ELMo.keep_prob: 1.0,
                                                                hyperQA_ELMo.query_glove:aa1_new,
                                                                hyperQA_ELMo.query_elmo:aa3_new,
                                                                hyperQA_ELMo.query_bmask:aa4_new,
                                                                hyperQA_ELMo.query_elmo_tokens:aa5_new,
                                                                hyperQA_ELMo.neg_ans_glove:bb1,
                                                                hyperQA_ELMo.neg_ans_elmo:bb3,
                                                                hyperQA_ELMo.neg_ans_bmask:bb4,
                                                                hyperQA_ELMo.neg_ans_elmo_tokens:bb5})
            result = result[0]; # array of batch size i.e. (BS,1)
            k=0;
            for n_times in dummy_batch_neg_n: # because no of neg answers per query is variable
              batch_arg_neg.append(np.argmax(result[k:k+n_times]));
              k+=n_times;
            unfort_index+=3;
      # TRAIN: compute loss and optimize
      if sampling_choice=='MAX':
        assert(len(batch_arg_neg)==len(batch_query_ids))
      for i_id, query_id in enumerate(batch_query_ids):
        batch_queries.append(data_dict[query_id]['query']);
        batch_pos.append(id2passage[data_dict[query_id]['pos']]);
        this_negative = id2passage[data_dict[query_id]['negs'][batch_arg_neg[i_id]]] if sampling_choice=='MAX' else id2passage[np.random.choice(data_dict[query_id]['negs'])];
        batch_neg.append(this_negative);
      #glove_arr, glove_bmasks, elmo_arr, elmo_bmasks, n_tokens = make_glove_and_elmo_batch_data(test_batch)
      aa1, _, aa3, aa4, aa5 = make_glove_and_elmo_batch_data(batch_queries,infer_max_len=True)
      bb1, _, bb3, bb4, bb5 = make_glove_and_elmo_batch_data(batch_pos,infer_max_len=True) 
      cc1, _, cc3, cc4, cc5 = make_glove_and_elmo_batch_data(batch_neg,infer_max_len=True)
      result = sess.run([hyperQA_ELMo.train_op, hyperQA_ELMo.print_losses, hyperQA_ELMo.print_p_score_loss, hyperQA_ELMo.print_n_score_loss],
                        feed_dict={hyperQA_ELMo.keep_prob: 0.8,
                                   hyperQA_ELMo.reg_lambda: 0.00001,
                                   hyperQA_ELMo.query_glove:aa1,
                                   hyperQA_ELMo.pos_ans_glove:bb1,
                                   hyperQA_ELMo.neg_ans_glove:cc1,
                                   hyperQA_ELMo.query_elmo:aa3,
                                   hyperQA_ELMo.query_elmo_tokens:aa5,
                                   hyperQA_ELMo.pos_ans_elmo:bb3,
                                   hyperQA_ELMo.pos_ans_elmo_tokens:bb5,
                                   hyperQA_ELMo.neg_ans_elmo:cc3,
                                   hyperQA_ELMo.neg_ans_elmo_tokens:cc5,
                                   hyperQA_ELMo.query_bmask:aa4,
                                   hyperQA_ELMo.pos_ans_bmask:bb4,
                                   hyperQA_ELMo.neg_ans_bmask:cc4})    
      cum_loss+=result[1];
      cum_pos_loss+=result[2];
      cum_neg_loss+=result[3];
      frm+=TRAINING_BATCH_SIZE;
      progressBar(frm,n_train,['loss','pos_loss','neg_loss'],[result[1],result[2],result[3]]);
    print('\nTRAINING || mean loss: {} | mean pos loss: {} | mean neg loss: {}'.format(cum_loss/n_batches,cum_pos_loss/n_batches,cum_neg_loss/n_batches))
    saver.save(sess, './'+model_save_folder+'/epoch_model_{}.ckpt'.format(epoch));
    # VALIDATE
    if epoch!=0 and epoch%VALIDATION_EVERY==0:
      #saver = tf.train.Saver();
      #saver.restore(val_sess, './'+model_save_folder+'/epoch_model_20.ckpt');
      n_batches = int(np.floor(n_val/VALIDATION_BATCH_SIZE));
      cum_loss = 0;
      cum_pos_loss = 0;
      cum_neg_loss = 0;
      frm=0;
      for i in range(n_batches):
        batch_query_ids = validation_query_ids[frm:frm+VALIDATION_BATCH_SIZE];
        batch_queries = [];
        batch_pos = [];
        batch_neg = [];
        batch_arg_neg = [];
        # VALIDATE: Find the most challenging negative!
        if sampling_choice=='MAX': #get j's which are maximums in their negative sets
            unfort_index = 0;
            while(unfort_index<len(batch_query_ids)):
                small_batch_query_ids = batch_query_ids[unfort_index:np.min([unfort_index+3,len(batch_query_ids)])];
                dummy_batch_queries = [];
                dummy_batch_neg = [];
                dummy_batch_neg_n = [];
                for query_id in small_batch_query_ids:
                  dummy_batch_queries.append(data_dict[query_id]['query']);
                  for jj in data_dict[query_id]['negs']:
                    dummy_batch_neg.append(id2passage[jj]);
                  dummy_batch_neg_n.append(len(data_dict[query_id]['negs']))        
                #glove_arr, glove_bmasks, elmo_arr, elmo_bmasks, n_tokens = make_glove_and_elmo_batch_data(test_batch)
                aa1, _, aa3, aa4, aa5 = make_glove_and_elmo_batch_data(dummy_batch_queries,infer_max_len=True)
                bb1, _, bb3, bb4, bb5 = make_glove_and_elmo_batch_data(dummy_batch_neg,infer_max_len=True)
                aa1_new, aa3_new, aa4_new, aa5_new = [], [], [], [];
                for ii,n_times in enumerate(dummy_batch_neg_n): # to tile x and y for thses many times 
                  aa1_new.append(np.tile(aa1[ii],(n_times,1)));
                  aa3_new.append(np.tile(aa3[ii],(n_times,1)));
                  aa4_new.append(np.tile(aa4[ii],(n_times,1)));
                  aa5_new.append(np.tile(aa5[ii],(n_times,1)));
                aa1_new, aa3_new, aa4_new, aa5_new = np.vstack(aa1_new), np.vstack(aa3_new), np.vstack(aa4_new), np.vstack(aa5_new);
                aa5_new = np.squeeze(aa5_new) # making it 1-dim
                result = sess.run([hyperQA_ELMo.n_score],feed_dict={hyperQA_ELMo.keep_prob: 1.0,
                                                                    hyperQA_ELMo.query_glove:aa1_new,
                                                                    hyperQA_ELMo.query_elmo:aa3_new,
                                                                    hyperQA_ELMo.query_bmask:aa4_new,
                                                                    hyperQA_ELMo.query_elmo_tokens:aa5_new,
                                                                    hyperQA_ELMo.neg_ans_glove:bb1,
                                                                    hyperQA_ELMo.neg_ans_elmo:bb3,
                                                                    hyperQA_ELMo.neg_ans_bmask:bb4,
                                                                    hyperQA_ELMo.neg_ans_elmo_tokens:bb5})
                result = result[0]; # array of batch size i.e. (BS,1)
                k=0;
                for n_times in dummy_batch_neg_n: # because no of neg answers per query is variable
                  batch_arg_neg.append(np.argmax(result[k:k+n_times]));
                  k+=n_times;
                unfort_index+=3;
        # VALIDATE: compute loss only
        if sampling_choice=='MAX':
          assert(len(batch_arg_neg)==len(batch_query_ids))
        for i_id, query_id in enumerate(batch_query_ids):
          batch_queries.append(data_dict[query_id]['query']);
          batch_pos.append(id2passage[data_dict[query_id]['pos']]);
          this_negative = id2passage[data_dict[query_id]['negs'][batch_arg_neg[i_id]]] if sampling_choice=='MAX' else id2passage[np.random.choice(data_dict[query_id]['negs'])];
          batch_neg.append(this_negative);
        #glove_arr, glove_bmasks, elmo_arr, elmo_bmasks, n_tokens = make_glove_and_elmo_batch_data(test_batch)
        aa1, _, aa3, aa4, aa5 = make_glove_and_elmo_batch_data(batch_queries,infer_max_len=True)
        bb1, _, bb3, bb4, bb5 = make_glove_and_elmo_batch_data(batch_pos,infer_max_len=True) 
        cc1, _, cc3, cc4, cc5 = make_glove_and_elmo_batch_data(batch_neg,infer_max_len=True)
        result = sess.run([hyperQA_ELMo.print_losses, hyperQA_ELMo.print_p_score_loss, hyperQA_ELMo.print_n_score_loss],
                          feed_dict={hyperQA_ELMo.keep_prob: 0.8,
                                     hyperQA_ELMo.reg_lambda: 0.00001,
                                     hyperQA_ELMo.query_glove:aa1,
                                     hyperQA_ELMo.pos_ans_glove:bb1,
                                     hyperQA_ELMo.neg_ans_glove:cc1,
                                     hyperQA_ELMo.query_elmo:aa3,
                                     hyperQA_ELMo.query_elmo_tokens:aa5,
                                     hyperQA_ELMo.pos_ans_elmo:bb3,
                                     hyperQA_ELMo.pos_ans_elmo_tokens:bb5,
                                     hyperQA_ELMo.neg_ans_elmo:cc3,
                                     hyperQA_ELMo.neg_ans_elmo_tokens:cc5,
                                     hyperQA_ELMo.query_bmask:aa4,
                                     hyperQA_ELMo.pos_ans_bmask:bb4,
                                     hyperQA_ELMo.neg_ans_bmask:cc4})  
        cum_loss+=result[0];
        cum_pos_loss+=result[1];
        cum_neg_loss+=result[2];
        frm+=VALIDATION_BATCH_SIZE;
        progressBar(frm,n_val,['loss','pos_loss','neg_loss'],[result[0],result[1],result[2]]);
      print('\nVALIDATION || mean loss: {} | mean pos loss: {} | mean neg loss: {}'.format(cum_loss/n_batches,cum_pos_loss/n_batches,cum_neg_loss/n_batches))

In [ ]:
'''
tf.reset_default_graph();
elmo = hub.Module("elmo2", trainable=True)
#def ElmoEmbedding(x_tf): #x_tf is a tensor 
#    return elmo(x_tf, as_dict = True, signature='default')['elmo']
def ElmoEmbedding(tokens_input, tokens_length): #inputs here are tensors 
    return elmo(inputs={"tokens": tokens_input,"sequence_len": tokens_length},signature="tokens",as_dict=True)["elmo"]
    
#embeddings = elmo(["the cat is on the mat", "dogs are in the fog"],signature="default",as_dict=True)["elmo"]
#embeddings

test_batch = ['How are you doing?','What is your age?', 'How to kill a cockroach..damn'];
glove_arr, glove_bmasks, elmo_arr, elmo_bmasks, n_tokens = make_glove_and_elmo_batch_data(test_batch) 
for item in [glove_arr, glove_bmasks, elmo_arr, elmo_bmasks, n_tokens]:
    print(item)
'''

In [ ]:
'''
glove_inp = tf.placeholder(tf.int32, shape=(None, None))
elmo_inp = tf.placeholder(tf.string, shape=(None, None))
elmo_token_len = tf.placeholder(tf.int32, shape=(None))

embedding_tf = tf.Variable(dfg.values , trainable=False, dtype=tf.float32, name='embeddings')
glove_emb = tf.nn.embedding_lookup(embedding_tf, glove_inp) # [None,None,emb_dim]
elmo_emb = ElmoEmbedding(elmo_inp, elmo_token_len);
final_emb = tf.concat([glove_emb,elmo_emb],axis=-1,name='concat')

sess = tf.Session(config=tf_sess_config);
sess.run(tf.global_variables_initializer());
sess.run(tf.tables_initializer());
check = sess.run([final_emb,glove_emb,elmo_emb],feed_dict={glove_inp:glove_arr,elmo_inp:elmo_arr,elmo_token_len:n_tokens})
'''

In [23]:
# args
EVAL_BATCH_SIZE = 32;

# load data
df_test = pd.read_csv("eval1_unlabelled.tsv", sep= '\t', header=None)
print('Eval Data Loaded.')

Eval Data Loaded.


In [26]:
# get scores
outputs = [];
with tf.Session(config=tf_sess_config) as sess:
  saver = tf.train.Saver();
  saver.restore(sess, r'./'+model_save_folder+'/epoch_model_1.ckpt');
  for i, row in df_test.iterrows():
    if i==0:
      batch_queries = [];
      batch_passages = [];
    if i!=0 and i%EVAL_BATCH_SIZE==0:
      #glove_arr, glove_bmasks, elmo_arr, elmo_bmasks, n_tokens = make_glove_and_elmo_batch_data(test_batch)
      aa1, _, aa3, aa4, aa5 = make_glove_and_elmo_batch_data(batch_queries,infer_max_len=True)
      bb1, _, bb3, bb4, bb5 = make_glove_and_elmo_batch_data(batch_passages,infer_max_len=True)
      result = sess.run([hyperQA_ELMo.p_score],feed_dict={hyperQA_ELMo.keep_prob: 1.0,
                                                          hyperQA_ELMo.query_glove:aa1,
                                                          hyperQA_ELMo.pos_ans_glove:bb1,
                                                          hyperQA_ELMo.query_elmo:aa3,
                                                          hyperQA_ELMo.query_elmo_tokens:aa5,
                                                          hyperQA_ELMo.pos_ans_elmo:bb3,
                                                          hyperQA_ELMo.pos_ans_elmo_tokens:bb5,
                                                          hyperQA_ELMo.query_bmask:aa4,
                                                          hyperQA_ELMo.pos_ans_bmask:bb4})
      outputs.append(result[0].tolist());
      progressBar(i,df_test.shape[0]);
      batch_queries = [];
      batch_passages = [];
    query, passage = row[1], row[2];
    batch_queries.append(query);
    batch_passages.append(passage);
  if batch_queries:
    #glove_arr, glove_bmasks, elmo_arr, elmo_bmasks, n_tokens = make_glove_and_elmo_batch_data(test_batch)
    aa1, _, aa3, aa4, aa5 = make_glove_and_elmo_batch_data(batch_queries,infer_max_len=True)
    bb1, _, bb3, bb4, bb5 = make_glove_and_elmo_batch_data(batch_passages,infer_max_len=True)
    result = sess.run([hyperQA_ELMo.p_score],feed_dict={hyperQA_ELMo.keep_prob: 1.0,
                                                        hyperQA_ELMo.query_glove:aa1,
                                                        hyperQA_ELMo.pos_ans_glove:bb1,
                                                        hyperQA_ELMo.query_elmo:aa3,
                                                        hyperQA_ELMo.query_elmo_tokens:aa5,
                                                        hyperQA_ELMo.pos_ans_elmo:bb3,
                                                        hyperQA_ELMo.pos_ans_elmo_tokens:bb5,
                                                        hyperQA_ELMo.query_bmask:aa4,
                                                        hyperQA_ELMo.pos_ans_bmask:bb4})
    outputs.append(result[0].tolist());
    progressBar(i,df_test.shape[0]);
    batch_queries = [];
    batch_passages = [];
outputs = np.vstack(outputs)
df_test['pred'] = outputs; # the smaller the score, the more positive_answer it is

INFO:tensorflow:Restoring parameters from ./hyperQA_elmo_100/epoch_model_1.ckpt
Percent: [------------------->][104169/104170] 100% ||  : -1.000000 | 

In [27]:
# re-using code from Dileep
outfilename = 'answer.tsv'
with open(outfilename,"w",encoding="utf-8") as fw:
  import math
  linelist = []
  tempscores = []
  for idx, row in tqdm(df_test.iterrows()):
      tempscores.append(row['pred'])
      #tempscores.append(str(row['pred']))
      if((idx +1)%10==0):
          tempscores-=np.min(tempscores);
          tempscores = [math.exp(s) for s in tempscores];
          expsum = sum(tempscores)
          tempscores = [str(s/expsum) for s in tempscores]
          scoreString = "\t".join(tempscores)
          qid = str(row[0])
          fw.write(qid+"\t"+scoreString+"\n")
          tempscores=[]
      #if(idx%10000==0):
      #    print(idx)

104170it [00:07, 13759.19it/s]


# HyperQA Training from Scrath with ELMo

In [ ]:
import tensorflow_hub as hub

In [ ]:
# HyperQA Tensorflow model

class HyperQA_ELMo(object):
    def __init__(self, projection_dim, margin, lr=0.001, elmo=None, elmo_emb_dim=1024):

        # parameters and variables
        self.projection_dim = projection_dim;
        self.margin = margin;
        self.lr = lr;
        self.elmo = elmo;
        self.elmo_emb_dim = self.concat_dim = elmo_emb_dim;
        self.init_name = tf.contrib.layers.xavier_initializer()
        self.w_p = tf.get_variable("w_p", shape=[self.concat_dim,self.projection_dim], initializer=self.init_name, trainable=True, dtype=tf.float32)
        self.b_p = tf.get_variable('b_p', shape=[self.projection_dim], initializer=tf.zeros_initializer(), trainable=True, dtype=tf.float32)
        self.w_f = tf.get_variable("w_f", shape=[1], initializer=self.init_name, trainable=True, dtype=tf.float32)
        self.b_f = tf.get_variable('b_f', shape=[1], initializer=tf.zeros_initializer(), trainable=True, dtype=tf.float32)

        # placeholder: general
        self.keep_prob = tf.placeholder(tf.float32, [])
        self.reg_lambda = tf.placeholder(tf.float32, [])
        # placeholder: elmo
        self.query_elmo = tf.placeholder(tf.string, shape=(None, None));
        self.query_elmo_tokens = tf.placeholder(tf.int32, shape=(None));
        self.pos_ans_elmo = tf.placeholder(tf.string, shape=(None, None));
        self.pos_ans_elmo_tokens = tf.placeholder(tf.int32, shape=(None));
        self.neg_ans_elmo = tf.placeholder(tf.string, shape=(None, None));
        self.neg_ans_elmo_tokens = tf.placeholder(tf.int32, shape=(None));
        # placeholder: masking
        self.query_bmask = tf.placeholder(tf.float32, [None, None])
        self.pos_ans_bmask = tf.placeholder(tf.float32, [None, None])
        self.neg_ans_bmask = tf.placeholder(tf.float32, [None, None])

        # get elmo embeddings # [None,None,elmo_emb_dim]
        self.query_elmo_emb = self.ElmoEmbedding(self.query_elmo, self.query_elmo_tokens);
        self.pos_ans_elmo_emb = self.ElmoEmbedding(self.pos_ans_elmo, self.pos_ans_elmo_tokens);
        self.neg_ans_elmo_emb = self.ElmoEmbedding(self.neg_ans_elmo, self.neg_ans_elmo_tokens);

        # concat embeddings # [None,None,concat_dim]
        self.query_emb = self.query_elmo_emb
        self.pos_ans_emb = self.pos_ans_elmo_emb
        self.neg_ans_emb = self.neg_ans_elmo_emb

        # projections with input # [None,None,glove_emb_dim] and output # [None,None,proj_emb]
        self.query_proj_ = self.project_fn(self.query_emb, self.query_bmask);
        self.pos_ans_proj_ = self.project_fn(self.pos_ans_emb, self.pos_ans_bmask);
        self.neg_ans_proj_ = self.project_fn(self.neg_ans_emb, self.neg_ans_bmask);
        # dropout
        self.query_proj = tf.nn.dropout(self.query_proj_, self.keep_prob);
        self.pos_ans_proj = tf.nn.dropout(self.pos_ans_proj_, self.keep_prob);
        self.neg_ans_proj = tf.nn.dropout(self.neg_ans_proj_, self.keep_prob);
        # unit normalized representations with output #[None,proj_emb]
        self.query_vec = tf.clip_by_norm(tf.reduce_sum(self.query_proj,axis=1), 1.0, axes=1) 
        self.pos_ans_vec = tf.clip_by_norm(tf.reduce_sum(self.pos_ans_proj,axis=1), 1.0, axes=1)
        self.neg_ans_vec = tf.clip_by_norm(tf.reduce_sum(self.neg_ans_proj,axis=1), 1.0, axes=1)
        # hyperbolic distance
        self.p_distance = self.hyperbolic_ball(self.query_vec, self.pos_ans_vec) #[None,1]
        self.n_distance = self.hyperbolic_ball(self.query_vec, self.neg_ans_vec) #[None,1]
        # loss
        self.p_score = self.p_distance*self.w_f+self.b_f; 
        self.n_score = self.n_distance*self.w_f+self.b_f;
        self.losses = tf.nn.relu(self.margin + self.n_score - self.p_score) #[None,1]
        self.reg_losses = self.reg_lambda*tf.reduce_sum(tf.abs(self.w_p));
        self.loss = self.reg_losses+tf.reduce_sum(self.losses) #[]
        # print loss ops
        self.print_p_distance = tf.reduce_mean(self.p_distance)
        self.print_n_distance = tf.reduce_mean(self.n_distance)
        self.print_p_score_loss = tf.reduce_mean(self.p_score)
        self.print_n_score_loss = tf.reduce_mean(self.n_score)
        self.print_losses = tf.reduce_mean(self.losses)
        # optimizer
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.lr)
        # adjust gradient
        gradients = self.optimizer.compute_gradients(self.loss)
        reim_gradients = [(self._to_riemannian_gradient(grad), var) for grad, var in gradients]
        clip_gradients = [(self._ClipIfNotNone(grad), var) for grad, var in reim_gradients]
        self.train_op = self.optimizer.apply_gradients(clip_gradients)  

    def ElmoEmbedding(self, tokens_input, tokens_length): #inputs here are tensors 
        return self.elmo(inputs={"tokens": tokens_input,"sequence_len": tokens_length},signature="tokens",as_dict=True)["elmo"]     
    def project_fn(self, inp_emb, bmask): # Input Shape [None,None,concat_dim], [None,None]
        runtime_shape = tf.shape(inp_emb);
        dim1 = runtime_shape[0];
        dim2 = runtime_shape[1];
        dense_output = tf.nn.xw_plus_b(tf.reshape(inp_emb, [-1,self.concat_dim]), self.w_p, self.b_p);
        activated_output = tf.nn.relu(dense_output);
        proj_emb = tf.reshape(activated_output,[dim1,dim2,self.projection_dim]);
        bmask = tf.tile(tf.expand_dims(bmask,axis=-1),[1,1,self.projection_dim]);
        masked_proj_emb = bmask*proj_emb;
        return masked_proj_emb
    def hyperbolic_ball(self, x, y, neg=False, eps=1E-6):
        """ Poincare Distance Function """
        z = x - y
        z = tf.norm(z, ord='euclidean', keep_dims=True, axis=1)
        z = tf.square(z)
        x_d = 1 - tf.square(tf.norm(x, ord='euclidean', keep_dims=True, axis=1))
        y_d = 1 - tf.square(tf.norm(y, ord='euclidean', keep_dims=True, axis=1))
        d = x_d * y_d
        z = z / (d + eps)
        z  = (2 * z) + 1
        arcosh = z + tf.sqrt(tf.square(z) - 1 + eps)
        arcosh = tf.log(arcosh)
        if(neg):
            arcosh = -arcosh
        return arcosh
    def _ClipIfNotNone(self, grad):
        if grad is None:
          return grad
        grad = tf.clip_by_value(grad, -10, 10, name=None)
        #grad = tf.clip_by_norm(grad, 1.0)
        return grad
    def _to_riemannian_gradient(self, ge):
      if ge is None:
        return None
      try:
        shape = ge.get_shape().as_list()
        if len(shape) >= 3:
            grad_scale = 1 - tf.square(tf.norm(ge, axis=[-2, -1], keepdims=True))
        elif len(shape) == 2:
            grad_scale = 1 - tf.square(tf.norm(ge, keepdims=True))
        else:
            return ge
      except:
        print('Exception handled!')
        grad_scale = 1 - tf.square(tf.norm(ge, keep_dims=True))
      grad_scale = (tf.square(grad_scale) + 1e-10) / 4.0
      gr = ge * grad_scale
      # gr = tf.clip_by_norm(gr, 1.0, axes=0)
      return gr

In [ ]:
# some global vars
PROJ_DIMS = 100;
MARGIN = 5.0;
LEARNING_RATE = 0.001;
TRAINING_BATCH_SIZE = 32;
VALIDATION_BATCH_SIZE = 32;
VALIDATION_EVERY = 1;
N_EPOCHS = 100;

# Create a graph with given params
tf.reset_default_graph();
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
hyperQA_ELMo = HyperQA_ELMo(PROJ_DIMS, MARGIN, LEARNING_RATE, elmo=elmo, elmo_emb_dim=1024)
print('graph built')

In [ ]:
cleanText = cleanText_Spaces_;

In [ ]:
# NEW: max sampling from https://cs.uwaterloo.ca/~jimmylin/publications/Rao_etal_CIKM2016.pdf
# Run this cell for both random_ as well as max_sampling
choice_max = 0; # the greater this is, the more cprob to chosse MAX over RANDOM
default_sampling_choice = 'RANDOM';
RESUME_TRAINING = False;
START_EPOCH = 0;

# Run the connections for desired epochs
with tf.Session() as sess:
  saver = tf.train.Saver();
  if not RESUME_TRAINING:
    sess.run(tf.global_variables_initializer());
    START_EPOCH = 0;
  else:
    saver.restore(sess, './'+model_save_folder+'/epoch_model_{}.ckpt'.format(START_EPOCH-1));
  for epoch in np.arange(START_EPOCH, N_EPOCHS):
    # sampling choice
    if epoch==0:
      sampling_choice = default_sampling_choice;
    else:
      sampling_choice = 'MAX' if np.random.random()<choice_max else default_sampling_choice;
    # Loss and optimization on Training Data
    print('beginning epoch: {}'.format(epoch));
    n_batches = int(np.floor(n_train/TRAINING_BATCH_SIZE));
    cum_loss = 0;
    cum_pos_loss = 0;
    cum_neg_loss = 0;
    frm=0;
    # TRAIN
    for i in range(n_batches):
      if i%1000==0:
        saver.save(sess, './'+model_save_folder+'/batch_model.ckpt'.format(epoch));
      batch_query_ids = training_query_ids[frm:frm+TRAINING_BATCH_SIZE];
      batch_queries = [];
      batch_pos = [];
      batch_neg = [];
      batch_arg_neg = [];
      # TRAIN: Find the most challenging negative!
      if sampling_choice=='MAX': #get j's which are maximums in their negative sets
        unfort_index = 0;
        while(unfort_index<len(batch_query_ids)):
            small_batch_query_ids = batch_query_ids[unfort_index:np.min([unfort_index+3,len(batch_query_ids)])];
            dummy_batch_queries = [];
            dummy_batch_neg = [];
            dummy_batch_neg_n = [];
            for query_id in small_batch_query_ids:
              dummy_batch_queries.append(data_dict[query_id]['query']);
              for jj in data_dict[query_id]['negs']:
                dummy_batch_neg.append(id2passage[jj]);
              dummy_batch_neg_n.append(len(data_dict[query_id]['negs']))        
            #glove_arr, glove_bmasks, elmo_arr, elmo_bmasks, n_tokens = make_glove_and_elmo_batch_data(test_batch)
            aa1, _, aa3, aa4, aa5 = make_glove_and_elmo_batch_data(dummy_batch_queries,infer_max_len=True)
            bb1, _, bb3, bb4, bb5 = make_glove_and_elmo_batch_data(dummy_batch_neg,infer_max_len=True)
            aa1_new, aa3_new, aa4_new, aa5_new = [], [], [], [];
            for ii,n_times in enumerate(dummy_batch_neg_n): # to tile x and y for thses many times 
              aa1_new.append(np.tile(aa1[ii],(n_times,1)));
              aa3_new.append(np.tile(aa3[ii],(n_times,1)));
              aa4_new.append(np.tile(aa4[ii],(n_times,1)));
              aa5_new.append(np.tile(aa5[ii],(n_times,1)));
            aa1_new, aa3_new, aa4_new, aa5_new = np.vstack(aa1_new), np.vstack(aa3_new), np.vstack(aa4_new), np.vstack(aa5_new);
            aa5_new = np.squeeze(aa5_new) # making it 1-dim
            result = sess.run([hyperQA_ELMo.n_score],feed_dict={hyperQA_ELMo.keep_prob: 1.0,
                                                                hyperQA_ELMo.query_glove:aa1_new,
                                                                hyperQA_ELMo.query_elmo:aa3_new,
                                                                hyperQA_ELMo.query_bmask:aa4_new,
                                                                hyperQA_ELMo.query_elmo_tokens:aa5_new,
                                                                hyperQA_ELMo.neg_ans_glove:bb1,
                                                                hyperQA_ELMo.neg_ans_elmo:bb3,
                                                                hyperQA_ELMo.neg_ans_bmask:bb4,
                                                                hyperQA_ELMo.neg_ans_elmo_tokens:bb5})
            result = result[0]; # array of batch size i.e. (BS,1)
            k=0;
            for n_times in dummy_batch_neg_n: # because no of neg answers per query is variable
              batch_arg_neg.append(np.argmax(result[k:k+n_times]));
              k+=n_times;
            unfort_index+=3;
      # TRAIN: compute loss and optimize
      if sampling_choice=='MAX':
        assert(len(batch_arg_neg)==len(batch_query_ids))
      for i_id, query_id in enumerate(batch_query_ids):
        batch_queries.append(data_dict[query_id]['query']);
        batch_pos.append(id2passage[data_dict[query_id]['pos']]);
        this_negative = id2passage[data_dict[query_id]['negs'][batch_arg_neg[i_id]]] if sampling_choice=='MAX' else id2passage[np.random.choice(data_dict[query_id]['negs'])];
        batch_neg.append(this_negative);
      #glove_arr, glove_bmasks, elmo_arr, elmo_bmasks, n_tokens = make_glove_and_elmo_batch_data(test_batch)
      aa3, aa4, aa5 = make_elmo_batch_data(batch_queries,infer_max_len=True)
      bb3, bb4, bb5 = make_elmo_batch_data(batch_pos,infer_max_len=True) 
      cc3, cc4, cc5 = make_elmo_batch_data(batch_neg,infer_max_len=True)
      result = sess.run([hyperQA_ELMo.train_op, hyperQA_ELMo.print_losses, hyperQA_ELMo.print_p_score_loss, hyperQA_ELMo.print_n_score_loss],
                        feed_dict={hyperQA_ELMo.keep_prob: 0.8,
                                   hyperQA_ELMo.reg_lambda: 0.00001,
                                   hyperQA_ELMo.query_elmo:aa3,
                                   hyperQA_ELMo.query_elmo_tokens:aa5,
                                   hyperQA_ELMo.pos_ans_elmo:bb3,
                                   hyperQA_ELMo.pos_ans_elmo_tokens:bb5,
                                   hyperQA_ELMo.neg_ans_elmo:cc3,
                                   hyperQA_ELMo.neg_ans_elmo_tokens:cc5,
                                   hyperQA_ELMo.query_bmask:aa4,
                                   hyperQA_ELMo.pos_ans_bmask:bb4,
                                   hyperQA_ELMo.neg_ans_bmask:cc4})    
      cum_loss+=result[1];
      cum_pos_loss+=result[2];
      cum_neg_loss+=result[3];
      frm+=TRAINING_BATCH_SIZE;
      progressBar(frm,n_train,['loss','pos_loss','neg_loss'],[result[1],result[2],result[3]]);
    print('\nTRAINING || mean loss: {} | mean pos loss: {} | mean neg loss: {}'.format(cum_loss/n_batches,cum_pos_loss/n_batches,cum_neg_loss/n_batches))
    saver.save(sess, './'+model_save_folder+'/epoch_model_{}.ckpt'.format(epoch));
    # VALIDATE
    if epoch!=0 and epoch%VALIDATION_EVERY==0:
      #saver = tf.train.Saver();
      #saver.restore(val_sess, './'+model_save_folder+'/epoch_model_20.ckpt');
      n_batches = int(np.floor(n_val/VALIDATION_BATCH_SIZE));
      cum_loss = 0;
      cum_pos_loss = 0;
      cum_neg_loss = 0;
      frm=0;
      for i in range(n_batches):
        batch_query_ids = validation_query_ids[frm:frm+VALIDATION_BATCH_SIZE];
        batch_queries = [];
        batch_pos = [];
        batch_neg = [];
        batch_arg_neg = [];
        # VALIDATE: Find the most challenging negative!
        if sampling_choice=='MAX': #get j's which are maximums in their negative sets
            unfort_index = 0;
            while(unfort_index<len(batch_query_ids)):
                small_batch_query_ids = batch_query_ids[unfort_index:np.min([unfort_index+3,len(batch_query_ids)])];
                dummy_batch_queries = [];
                dummy_batch_neg = [];
                dummy_batch_neg_n = [];
                for query_id in small_batch_query_ids:
                  dummy_batch_queries.append(data_dict[query_id]['query']);
                  for jj in data_dict[query_id]['negs']:
                    dummy_batch_neg.append(id2passage[jj]);
                  dummy_batch_neg_n.append(len(data_dict[query_id]['negs']))        
                #glove_arr, glove_bmasks, elmo_arr, elmo_bmasks, n_tokens = make_glove_and_elmo_batch_data(test_batch)
                aa1, _, aa3, aa4, aa5 = make_glove_and_elmo_batch_data(dummy_batch_queries,infer_max_len=True)
                bb1, _, bb3, bb4, bb5 = make_glove_and_elmo_batch_data(dummy_batch_neg,infer_max_len=True)
                aa1_new, aa3_new, aa4_new, aa5_new = [], [], [], [];
                for ii,n_times in enumerate(dummy_batch_neg_n): # to tile x and y for thses many times 
                  aa1_new.append(np.tile(aa1[ii],(n_times,1)));
                  aa3_new.append(np.tile(aa3[ii],(n_times,1)));
                  aa4_new.append(np.tile(aa4[ii],(n_times,1)));
                  aa5_new.append(np.tile(aa5[ii],(n_times,1)));
                aa1_new, aa3_new, aa4_new, aa5_new = np.vstack(aa1_new), np.vstack(aa3_new), np.vstack(aa4_new), np.vstack(aa5_new);
                aa5_new = np.squeeze(aa5_new) # making it 1-dim
                result = sess.run([hyperQA_ELMo.n_score],feed_dict={hyperQA_ELMo.keep_prob: 1.0,
                                                                    hyperQA_ELMo.query_glove:aa1_new,
                                                                    hyperQA_ELMo.query_elmo:aa3_new,
                                                                    hyperQA_ELMo.query_bmask:aa4_new,
                                                                    hyperQA_ELMo.query_elmo_tokens:aa5_new,
                                                                    hyperQA_ELMo.neg_ans_glove:bb1,
                                                                    hyperQA_ELMo.neg_ans_elmo:bb3,
                                                                    hyperQA_ELMo.neg_ans_bmask:bb4,
                                                                    hyperQA_ELMo.neg_ans_elmo_tokens:bb5})
                result = result[0]; # array of batch size i.e. (BS,1)
                k=0;
                for n_times in dummy_batch_neg_n: # because no of neg answers per query is variable
                  batch_arg_neg.append(np.argmax(result[k:k+n_times]));
                  k+=n_times;
                unfort_index+=3;
        # VALIDATE: compute loss only
        if sampling_choice=='MAX':
          assert(len(batch_arg_neg)==len(batch_query_ids))
        for i_id, query_id in enumerate(batch_query_ids):
          batch_queries.append(data_dict[query_id]['query']);
          batch_pos.append(id2passage[data_dict[query_id]['pos']]);
          this_negative = id2passage[data_dict[query_id]['negs'][batch_arg_neg[i_id]]] if sampling_choice=='MAX' else id2passage[np.random.choice(data_dict[query_id]['negs'])];
          batch_neg.append(this_negative);
        #glove_arr, glove_bmasks, elmo_arr, elmo_bmasks, n_tokens = make_glove_and_elmo_batch_data(test_batch)
        aa3, aa4, aa5 = make_glove_and_elmo_batch_data(batch_queries,infer_max_len=True)
        bb3, bb4, bb5 = make_glove_and_elmo_batch_data(batch_pos,infer_max_len=True) 
        cc3, cc4, cc5 = make_glove_and_elmo_batch_data(batch_neg,infer_max_len=True)
        result = sess.run([hyperQA_ELMo.train_op, hyperQA_ELMo.print_losses, hyperQA_ELMo.print_p_score_loss, hyperQA_ELMo.print_n_score_loss],
                          feed_dict={hyperQA_ELMo.keep_prob: 0.8,
                                     hyperQA_ELMo.reg_lambda: 0.00001,
                                     hyperQA_ELMo.query_elmo:aa3,
                                     hyperQA_ELMo.query_elmo_tokens:aa5,
                                     hyperQA_ELMo.pos_ans_elmo:bb3,
                                     hyperQA_ELMo.pos_ans_elmo_tokens:bb5,
                                     hyperQA_ELMo.neg_ans_elmo:cc3,
                                     hyperQA_ELMo.neg_ans_elmo_tokens:cc5,
                                     hyperQA_ELMo.query_bmask:aa4,
                                     hyperQA_ELMo.pos_ans_bmask:bb4,
                                     hyperQA_ELMo.neg_ans_bmask:cc4})  
        cum_loss+=result[0];
        cum_pos_loss+=result[1];
        cum_neg_loss+=result[2];
        frm+=VALIDATION_BATCH_SIZE;
        progressBar(frm,n_val,['loss','pos_loss','neg_loss'],[result[0],result[1],result[2]]);
      print('\nVALIDATION || mean loss: {} | mean pos loss: {} | mean neg loss: {}'.format(cum_loss/n_batches,cum_pos_loss/n_batches,cum_neg_loss/n_batches))

In [ ]:
# args
EVAL_BATCH_SIZE = 32;


# In[ ]:

# load data
df_test = pd.read_csv("eval1_unlabelled.tsv", sep= '\t', header=None)
print('Eval Data Loaded.')

In [ ]:
# get scores
outputs = [];
with tf.Session(config=tf_sess_config) as sess:
  saver = tf.train.Saver();
  saver.restore(sess, r'./'+model_save_folder+'/epoch_model_0.ckpt');
  for i, row in df_test.iterrows():
    if i==0:
      batch_queries = [];
      batch_passages = [];
    if i!=0 and i%EVAL_BATCH_SIZE==0:
      #glove_arr, glove_bmasks, elmo_arr, elmo_bmasks, n_tokens = make_glove_and_elmo_batch_data(test_batch)
      aa1, _, aa3, aa4, aa5 = make_glove_and_elmo_batch_data(batch_queries,infer_max_len=True)
      bb1, _, bb3, bb4, bb5 = make_glove_and_elmo_batch_data(batch_passages,infer_max_len=True)
      result = sess.run([hyperQA_ELMo.p_score],feed_dict={hyperQA_ELMo.keep_prob: 1.0,
                                                          hyperQA_ELMo.query_glove:aa1,
                                                          hyperQA_ELMo.pos_ans_glove:bb1,
                                                          hyperQA_ELMo.query_elmo:aa3,
                                                          hyperQA_ELMo.query_elmo_tokens:aa5,
                                                          hyperQA_ELMo.pos_ans_elmo:bb3,
                                                          hyperQA_ELMo.pos_ans_elmo_tokens:bb5,
                                                          hyperQA_ELMo.query_bmask:aa4,
                                                          hyperQA_ELMo.pos_ans_bmask:bb4})
      outputs.append(result[0].tolist());
      progressBar(i,df_test.shape[0]);
      batch_queries = [];
      batch_passages = [];
    query, passage = row[1], row[2];
    batch_queries.append(query);
    batch_passages.append(passage);
  if batch_queries:
    #glove_arr, glove_bmasks, elmo_arr, elmo_bmasks, n_tokens = make_glove_and_elmo_batch_data(test_batch)
    aa1, _, aa3, aa4, aa5 = make_glove_and_elmo_batch_data(batch_queries,infer_max_len=True)
    bb1, _, bb3, bb4, bb5 = make_glove_and_elmo_batch_data(batch_passages,infer_max_len=True)
    result = sess.run([hyperQA_ELMo.p_score],feed_dict={hyperQA_ELMo.keep_prob: 1.0,
                                                        hyperQA_ELMo.query_glove:aa1,
                                                        hyperQA_ELMo.pos_ans_glove:bb1,
                                                        hyperQA_ELMo.query_elmo:aa3,
                                                        hyperQA_ELMo.query_elmo_tokens:aa5,
                                                        hyperQA_ELMo.pos_ans_elmo:bb3,
                                                        hyperQA_ELMo.pos_ans_elmo_tokens:bb5,
                                                        hyperQA_ELMo.query_bmask:aa4,
                                                        hyperQA_ELMo.pos_ans_bmask:bb4})
    outputs.append(result[0].tolist());
    progressBar(i,df_test.shape[0]);
    batch_queries = [];
    batch_passages = [];
outputs = np.vstack(outputs)
df_test['pred'] = outputs; # the smaller the score, the more positive_answer it is

In [ ]:
# re-using code from Dileep
outfilename = 'answer.tsv'
with open(outfilename,"w",encoding="utf-8") as fw:
  import math
  linelist = []
  tempscores = []
  for idx, row in tqdm(df_test.iterrows()):
      tempscores.append(row['pred'])
      #tempscores.append(str(row['pred']))
      if((idx +1)%10==0):
          tempscores-=np.min(tempscores);
          tempscores = [math.exp(s) for s in tempscores];
          expsum = sum(tempscores)
          tempscores = [str(s/expsum) for s in tempscores]
          scoreString = "\t".join(tempscores)
          qid = str(row[0])
          fw.write(qid+"\t"+scoreString+"\n")
          tempscores=[]
      #if(idx%10000==0):
      #    print(idx)

# BiLSTM-Attention-SiameseLSTM Model

In [32]:
# model save 
!mkdir biLSTM_Attention
model_save_folder = r'biLSTM_Attention'
dfile = open('./'+model_save_folder+'/dummy.txt','w',encoding='utf-8');
dfile.write('Checking...');
dfile.close();

In [28]:
# HyperQA Tensorflow model

class BiLSTM_Attention(object):
    def __init__(self, vocab_embedding, glove_emb_dim, margin, lr=0.001):

        # parameters and variables
        self.glove_embedding = tf.Variable(vocab_embedding, trainable=False, dtype=tf.float32, name='glove_embedding');
        self.glove_emb_dim = glove_emb_dim;
        self.margin = margin;
        self.lr = lr;

        # placeholder: general
        self.keep_prob = tf.placeholder(tf.float32, [])
        self.reg_lambda = tf.placeholder(tf.float32, [])
        # placeholder: glove
        self.query_glove = tf.placeholder(tf.int32, [None, None])
        self.pos_ans_glove = tf.placeholder(tf.int32, [None, None])
        self.neg_ans_glove = tf.placeholder(tf.int32, [None, None])
        # placeholder: tokens
        self.query_glove_seq_lengths = tf.placeholder(tf.int32, shape=(None));
        self.pos_ans_glove_seq_lengths = tf.placeholder(tf.int32, shape=(None));
        self.neg_ans_glove_seq_lengths = tf.placeholder(tf.int32, shape=(None));
        # placeholder: masking
        #self.query_bmask = tf.placeholder(tf.float32, [None, None])
        #self.pos_ans_bmask = tf.placeholder(tf.float32, [None, None])
        #self.neg_ans_bmask = tf.placeholder(tf.float32, [None, None])
        
        # layer: glove embedding # [None,None,glove_emb_dim]
        self.query_glove_emb = tf.nn.embedding_lookup(self.glove_embedding, self.query_glove)
        self.pos_ans_glove_emb = tf.nn.embedding_lookup(self.glove_embedding, self.pos_ans_glove)
        self.neg_ans_glove_emb = tf.nn.embedding_lookup(self.glove_embedding, self.neg_ans_glove)
        
        # layer: BiLSTM with 2 layers # time_major=False # [None,None,cell_fw.ouput_size+cell_bw.output_size]
        with tf.variable_scope("query_BiLSTM"):
          self.query_BiLSTM = self.query_BiLSTM_fn(self.query_glove_emb,self.query_glove_seq_lengths)
        with tf.variable_scope("answer_BiLSTM", reuse = tf.AUTO_REUSE):
          self.pos_ans_BiLSTM = self.answer_BiLSTM_fn(self.pos_ans_glove_emb,self.pos_ans_glove_seq_lengths)
          self.neg_ans_BiLSTM = self.answer_BiLSTM_fn(self.neg_ans_glove_emb,self.neg_ans_glove_seq_lengths)
 
        # layer: Attention+SiameseEncoder
        self.p_score = self.get_score_fn(self.query_BiLSTM, self.query_glove_seq_lengths, self.pos_ans_BiLSTM, self.pos_ans_glove_seq_lengths);
        self.n_score = self.get_score_fn(self.query_BiLSTM, self.query_glove_seq_lengths, self.neg_ans_BiLSTM, self.neg_ans_glove_seq_lengths);
      
        # layer: Loss and Optimization
        self.losses = tf.nn.relu(self.margin + self.n_score - self.p_score) #[None,1]
        #self.reg_losses = self.reg_lambda*tf.reduce_sum(tf.abs(self.w_p));
        self.loss = tf.reduce_sum(self.losses) # +self.reg_losses
        # optimizer
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.lr)
        # adjust gradient
        gradients = self.optimizer.compute_gradients(self.loss)
        clip_gradients = [(self._ClipIfNotNone(grad), var) for grad, var in gradients]
        self.train_op = self.optimizer.apply_gradients(clip_gradients)
        
        # print loss ops
        self.print_p_score_loss = tf.reduce_mean(self.p_score)
        self.print_n_score_loss = tf.reduce_mean(self.n_score)
        self.print_losses = tf.reduce_mean(self.losses)
    
    # Function Definitions
    def query_BiLSTM_fn(self, query_input, query_seq_lengths): # [None,None,glove_emb_dim]
      query_outputs_L1, query_output_states_L1 = tf.nn.bidirectional_dynamic_rnn(tf.nn.rnn_cell.LSTMCell(num_units=128),tf.nn.rnn_cell.LSTMCell(num_units=128),inputs=query_input,sequence_length=query_seq_lengths,dtype=tf.float32,scope='L1')
      query_outputs_concat_L1 = tf.concat(query_outputs_L1, 2)
      query_outputs_L2, query_output_states_L2 = tf.nn.bidirectional_dynamic_rnn(tf.nn.rnn_cell.LSTMCell(num_units=128),tf.nn.rnn_cell.LSTMCell(num_units=128),inputs=query_outputs_concat_L1,sequence_length=query_seq_lengths,dtype=tf.float32,scope='L2')
      query_outputs_concat_L2 = tf.concat(query_outputs_L2, 2)
      return query_outputs_concat_L2
    def answer_BiLSTM_fn(self, answer_input, answer_seq_lengths): # [None,None,glove_emb_dim]
      answer_outputs_L1, answer_output_states_L1 = tf.nn.bidirectional_dynamic_rnn(tf.nn.rnn_cell.LSTMCell(num_units=128),tf.nn.rnn_cell.LSTMCell(num_units=128),inputs=answer_input,sequence_length=answer_seq_lengths,dtype=tf.float32,scope='L1')
      answer_outputs_concat_L1 = tf.concat(answer_outputs_L1, 2)
      answer_outputs_L2, answer_output_states_L2 = tf.nn.bidirectional_dynamic_rnn(tf.nn.rnn_cell.LSTMCell(num_units=128),tf.nn.rnn_cell.LSTMCell(num_units=128),inputs=answer_outputs_concat_L1,sequence_length=answer_seq_lengths,dtype=tf.float32,scope='L2')
      answer_outputs_concat_L2 = tf.concat(answer_outputs_L2, 2)
      return answer_outputs_concat_L2
    def attention_fn(self, query, answer): # shape of dim1 in both inputs must be equal. # dim2 are time_axis and can be variable length
      query_time_steps, answer_time_steps = tf.shape(query)[1], tf.shape(answer)[1];
      # attention scores
      interaction_matrix = tf.matmul(query, tf.transpose(answer, perm=[0, 2, 1])); # shape [inputs_1.dims1, inputs_1.dims2, inputs_2.dims2]
      query_softmax_, answer_softmax_ = tf.nn.softmax(interaction_matrix,axis=2), tf.nn.softmax(interaction_matrix,axis=1)
      query_softmax, answer_softmax = tf.expand_dims(query_softmax_, axis=-1), tf.expand_dims(answer_softmax_, axis=-1)
      # query_attention
      query_expanded = tf.tile(tf.expand_dims(query,axis=-2), [1,1,answer_time_steps,1]);
      query_attn_ = query_expanded*query_softmax;
      query_attn = tf.reduce_sum(query_attn_, axis=-2);
      # answer_attention
      answer_expanded = tf.tile(tf.expand_dims(answer,axis=-3), [1,query_time_steps,1,1]);
      answer_attn_ = answer_expanded*answer_softmax;
      answer_attn = tf.reduce_sum(answer_attn_, axis=-3);
      # outputs
      output_query = tf.concat([query,query_attn], axis=-1);
      output_answer = tf.concat([answer,answer_attn], axis=-1);      
      return output_query, output_answer
    def siamese_BiLSTM_fn(self, input_seq, input_seq_lengths): # [None,None,2*(cell_fw.ouput_size+cell_bw.output_size)]
      outputs_L1, _ = tf.nn.bidirectional_dynamic_rnn(tf.nn.rnn_cell.LSTMCell(num_units=64),tf.nn.rnn_cell.LSTMCell(num_units=64),inputs=input_seq,sequence_length=input_seq_lengths,dtype=tf.float32,scope='L1')
      outputs_concat_L1 = tf.concat(outputs_L1, 2)
      return outputs_concat_L1    
    def extract_axis_1(self, data, ind):
      # https://stackoverflow.com/questions/41273361/get-the-last-output-of-a-dynamic-rnn-in-tensorflow
      batch_range = tf.range(tf.shape(data)[0])
      indices = tf.stack([batch_range, ind], axis=1)
      res = tf.gather_nd(data, indices)
      return res # [batch_size, num_cells]
    def get_score_fn(self, query, query_seq_lengths, answer, answer_seq_lengths):
      query_with_attn, answer_with_attn = self.attention_fn(query, answer);
      with tf.variable_scope("siamese_BiLSTM_encoder", reuse = tf.AUTO_REUSE):
        a1 = self.siamese_BiLSTM_fn(query_with_attn, query_seq_lengths);
        a2 = self.extract_axis_1(a1, query_seq_lengths-1)
        b1 = self.siamese_BiLSTM_fn(answer_with_attn, answer_seq_lengths);
        b2 = self.extract_axis_1(b1, answer_seq_lengths-1)
        c = tf.reduce_sum(tf.multiply(a2,b2), axis=-1);
      return c
    def _ClipIfNotNone(self, grad):
        if grad is None:
          return grad
        #grad = tf.clip_by_value(grad, -10, 10, name=None)
        grad = tf.clip_by_norm(grad, 5.0)
        return grad


In [29]:
# some global vars
MARGIN = 5.0;
LEARNING_RATE = 0.001;

# Create a graph with given params
tf.reset_default_graph();
biLSTM_Attention = BiLSTM_Attention(dfg.values, EMB_DIMS, MARGIN, LEARNING_RATE)
print('graph built')

Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead
graph built


In [30]:
cleanText = cleanText_Simple;

In [56]:
# NEW: max sampling from https://cs.uwaterloo.ca/~jimmylin/publications/Rao_etal_CIKM2016.pdf
# Run this cell for both random_ as well as max_sampling
choice_max = 0; # the greater this is, the more cprob to chosse MAX over RANDOM
default_sampling_choice = 'RANDOM';
RESUME_TRAINING = True;
START_EPOCH = 2;
TRAINING_BATCH_SIZE = 32;
VALIDATION_BATCH_SIZE = 64;
VALIDATION_EVERY = 10;
N_EPOCHS = 100;

# Run the connections for desired epochs
with tf.Session(config=tf_sess_config) as sess:
  saver = tf.train.Saver();
  if not RESUME_TRAINING:
    sess.run(tf.global_variables_initializer());
    START_EPOCH = 0;
  else:
    saver.restore(sess, './'+model_save_folder+'/epoch_model_{}.ckpt'.format(START_EPOCH-1));
  for epoch in np.arange(START_EPOCH, N_EPOCHS):
    # sampling choice
    if epoch==0:
      sampling_choice = default_sampling_choice;
    else:
      sampling_choice = 'MAX' if np.random.random()<choice_max else default_sampling_choice;
    # Loss and optimization on Training Data
    print('beginning epoch: {}'.format(epoch));
    n_batches = int(np.floor(n_train/TRAINING_BATCH_SIZE));
    cum_loss = 0;
    cum_pos_loss = 0;
    cum_neg_loss = 0;
    frm=0;
    # TRAIN
    for i in range(n_batches):
      if i%1000==0:
        saver.save(sess, './'+model_save_folder+'/batch_model.ckpt');
      batch_query_ids = training_query_ids[frm:frm+TRAINING_BATCH_SIZE];
      batch_queries = [];
      batch_pos = [];
      batch_neg = [];
      batch_arg_neg = [];
      # TRAIN: Find the most challenging negative!
      if sampling_choice=='MAX': #get j's which are maximums in their negative sets
        unfort_index = 0;
        while(unfort_index<len(batch_query_ids)):
            small_batch_query_ids = batch_query_ids[unfort_index:np.min([unfort_index+3,len(batch_query_ids)])];
            dummy_batch_queries = [];
            dummy_batch_neg = [];
            dummy_batch_neg_n = [];
            for query_id in small_batch_query_ids:
              dummy_batch_queries.append(data_dict[query_id]['query']);
              for jj in data_dict[query_id]['negs']:
                dummy_batch_neg.append(id2passage[jj]);
              dummy_batch_neg_n.append(len(data_dict[query_id]['negs']))        
            #glove_arr, glove_bmasks, elmo_arr, elmo_bmasks, n_tokens = make_glove_and_elmo_batch_data(test_batch)
            aa1, _, aa3, aa4, aa5 = make_glove_and_elmo_batch_data(dummy_batch_queries,infer_max_len=True)
            bb1, _, bb3, bb4, bb5 = make_glove_and_elmo_batch_data(dummy_batch_neg,infer_max_len=True)
            aa1_new, aa3_new, aa4_new, aa5_new = [], [], [], [];
            for ii,n_times in enumerate(dummy_batch_neg_n): # to tile x and y for thses many times 
              aa1_new.append(np.tile(aa1[ii],(n_times,1)));
              aa3_new.append(np.tile(aa3[ii],(n_times,1)));
              aa4_new.append(np.tile(aa4[ii],(n_times,1)));
              aa5_new.append(np.tile(aa5[ii],(n_times,1)));
            aa1_new, aa3_new, aa4_new, aa5_new = np.vstack(aa1_new), np.vstack(aa3_new), np.vstack(aa4_new), np.vstack(aa5_new);
            aa5_new = np.squeeze(aa5_new) # making it 1-dim
            result = sess.run([biLSTM_Attention.n_score],feed_dict={biLSTM_Attention.keep_prob: 1.0,
                                                                biLSTM_Attention.query_glove:aa1_new,
                                                                biLSTM_Attention.query_elmo:aa3_new,
                                                                biLSTM_Attention.query_bmask:aa4_new,
                                                                biLSTM_Attention.query_elmo_tokens:aa5_new,
                                                                biLSTM_Attention.neg_ans_glove:bb1,
                                                                biLSTM_Attention.neg_ans_elmo:bb3,
                                                                biLSTM_Attention.neg_ans_bmask:bb4,
                                                                biLSTM_Attention.neg_ans_elmo_tokens:bb5})
            result = result[0]; # array of batch size i.e. (BS,1)
            k=0;
            for n_times in dummy_batch_neg_n: # because no of neg answers per query is variable
              batch_arg_neg.append(np.argmax(result[k:k+n_times]));
              k+=n_times;
            unfort_index+=3;
      # TRAIN: compute loss and optimize
      if sampling_choice=='MAX':
        assert(len(batch_arg_neg)==len(batch_query_ids))
      for i_id, query_id in enumerate(batch_query_ids):
        batch_queries.append(data_dict[query_id]['query']);
        batch_pos.append(id2passage[data_dict[query_id]['pos']]);
        this_negative = id2passage[data_dict[query_id]['negs'][batch_arg_neg[i_id]]] if sampling_choice=='MAX' else id2passage[np.random.choice(data_dict[query_id]['negs'])];
        batch_neg.append(this_negative);
      #glove_arr, glove_bmasks, elmo_arr, elmo_bmasks, n_tokens = make_glove_and_elmo_batch_data(test_batch)
      #glove_arr, glove_bmasks, n_tokens = make_glove_batch_data(test_batch) 
      aa1, _, aa3 = make_glove_batch_data(batch_queries,infer_max_len=True)
      bb1, _, bb3 = make_glove_batch_data(batch_pos,infer_max_len=True) 
      cc1, _, cc3 = make_glove_batch_data(batch_neg,infer_max_len=True)
      result = sess.run([biLSTM_Attention.train_op, biLSTM_Attention.print_losses, biLSTM_Attention.print_p_score_loss, biLSTM_Attention.print_n_score_loss],
                        feed_dict={biLSTM_Attention.keep_prob: 0.8,
                                   biLSTM_Attention.reg_lambda: 0.00001,
                                   biLSTM_Attention.query_glove:aa1,
                                   biLSTM_Attention.pos_ans_glove:bb1,
                                   biLSTM_Attention.neg_ans_glove:cc1,
                                   biLSTM_Attention.query_glove_seq_lengths:aa3,
                                   biLSTM_Attention.pos_ans_glove_seq_lengths:bb3,
                                   biLSTM_Attention.neg_ans_glove_seq_lengths:cc3})    
      cum_loss+=result[1];
      cum_pos_loss+=result[2];
      cum_neg_loss+=result[3];
      frm+=TRAINING_BATCH_SIZE;
      progressBar(frm,n_train,['loss','pos_loss','neg_loss'],[result[1],result[2],result[3]]);
    print('\nTRAINING || mean loss: {} | mean pos loss: {} | mean neg loss: {}'.format(cum_loss/n_batches,cum_pos_loss/n_batches,cum_neg_loss/n_batches))
    saver.save(sess, './'+model_save_folder+'/epoch_model_{}.ckpt'.format(epoch));
    # VALIDATE
    if epoch!=0 and epoch%VALIDATION_EVERY==0:
      #saver = tf.train.Saver();
      #saver.restore(val_sess, './'+model_save_folder+'/epoch_model_20.ckpt');
      n_batches = int(np.floor(n_val/VALIDATION_BATCH_SIZE));
      cum_loss = 0;
      cum_pos_loss = 0;
      cum_neg_loss = 0;
      frm=0;
      for i in range(n_batches):
        batch_query_ids = validation_query_ids[frm:frm+VALIDATION_BATCH_SIZE];
        batch_queries = [];
        batch_pos = [];
        batch_neg = [];
        batch_arg_neg = [];
        # VALIDATE: Find the most challenging negative!
        if sampling_choice=='MAX': #get j's which are maximums in their negative sets
            unfort_index = 0;
            while(unfort_index<len(batch_query_ids)):
                small_batch_query_ids = batch_query_ids[unfort_index:np.min([unfort_index+3,len(batch_query_ids)])];
                dummy_batch_queries = [];
                dummy_batch_neg = [];
                dummy_batch_neg_n = [];
                for query_id in small_batch_query_ids:
                  dummy_batch_queries.append(data_dict[query_id]['query']);
                  for jj in data_dict[query_id]['negs']:
                    dummy_batch_neg.append(id2passage[jj]);
                  dummy_batch_neg_n.append(len(data_dict[query_id]['negs']))        
                #glove_arr, glove_bmasks, elmo_arr, elmo_bmasks, n_tokens = make_glove_and_elmo_batch_data(test_batch)
                aa1, _, aa3, aa4, aa5 = make_glove_and_elmo_batch_data(dummy_batch_queries,infer_max_len=True)
                bb1, _, bb3, bb4, bb5 = make_glove_and_elmo_batch_data(dummy_batch_neg,infer_max_len=True)
                aa1_new, aa3_new, aa4_new, aa5_new = [], [], [], [];
                for ii,n_times in enumerate(dummy_batch_neg_n): # to tile x and y for thses many times 
                  aa1_new.append(np.tile(aa1[ii],(n_times,1)));
                  aa3_new.append(np.tile(aa3[ii],(n_times,1)));
                  aa4_new.append(np.tile(aa4[ii],(n_times,1)));
                  aa5_new.append(np.tile(aa5[ii],(n_times,1)));
                aa1_new, aa3_new, aa4_new, aa5_new = np.vstack(aa1_new), np.vstack(aa3_new), np.vstack(aa4_new), np.vstack(aa5_new);
                aa5_new = np.squeeze(aa5_new) # making it 1-dim
                result = sess.run([biLSTM_Attention.n_score],feed_dict={biLSTM_Attention.keep_prob: 1.0,
                                                                    biLSTM_Attention.query_glove:aa1_new,
                                                                    biLSTM_Attention.query_elmo:aa3_new,
                                                                    biLSTM_Attention.query_bmask:aa4_new,
                                                                    biLSTM_Attention.query_elmo_tokens:aa5_new,
                                                                    biLSTM_Attention.neg_ans_glove:bb1,
                                                                    biLSTM_Attention.neg_ans_elmo:bb3,
                                                                    biLSTM_Attention.neg_ans_bmask:bb4,
                                                                    biLSTM_Attention.neg_ans_elmo_tokens:bb5})
                result = result[0]; # array of batch size i.e. (BS,1)
                k=0;
                for n_times in dummy_batch_neg_n: # because no of neg answers per query is variable
                  batch_arg_neg.append(np.argmax(result[k:k+n_times]));
                  k+=n_times;
                unfort_index+=3;
        # VALIDATE: compute loss only
        if sampling_choice=='MAX':
          assert(len(batch_arg_neg)==len(batch_query_ids))
        for i_id, query_id in enumerate(batch_query_ids):
          batch_queries.append(data_dict[query_id]['query']);
          batch_pos.append(id2passage[data_dict[query_id]['pos']]);
          this_negative = id2passage[data_dict[query_id]['negs'][batch_arg_neg[i_id]]] if sampling_choice=='MAX' else id2passage[np.random.choice(data_dict[query_id]['negs'])];
          batch_neg.append(this_negative);
        #glove_arr, glove_bmasks, elmo_arr, elmo_bmasks, n_tokens = make_glove_and_elmo_batch_data(test_batch)
        #glove_arr, glove_bmasks, n_tokens = make_glove_batch_data(test_batch) 
        aa1, _, aa3 = make_glove_batch_data(batch_queries,infer_max_len=True)
        bb1, _, bb3 = make_glove_batch_data(batch_pos,infer_max_len=True) 
        cc1, _, cc3 = make_glove_batch_data(batch_neg,infer_max_len=True)
        result = sess.run([biLSTM_Attention.print_losses, biLSTM_Attention.print_p_score_loss, biLSTM_Attention.print_n_score_loss],
                          feed_dict={biLSTM_Attention.keep_prob: 0.8,
                                     biLSTM_Attention.reg_lambda: 0.00001,
                                     biLSTM_Attention.query_glove:aa1,
                                     biLSTM_Attention.pos_ans_glove:bb1,
                                     biLSTM_Attention.neg_ans_glove:cc1,
                                     biLSTM_Attention.query_glove_seq_lengths:aa3,
                                     biLSTM_Attention.pos_ans_glove_seq_lengths:bb3,
                                     biLSTM_Attention.neg_ans_glove_seq_lengths:cc3}) 
        cum_loss+=result[0];
        cum_pos_loss+=result[1];
        cum_neg_loss+=result[2];
        frm+=VALIDATION_BATCH_SIZE;
        progressBar(frm,n_val,['loss','pos_loss','neg_loss'],[result[0],result[1],result[2]]);
      print('\nVALIDATION || mean loss: {} | mean pos loss: {} | mean neg loss: {}'.format(cum_loss/n_batches,cum_pos_loss/n_batches,cum_neg_loss/n_batches))

INFO:tensorflow:Restoring parameters from ./biLSTM_Attention/epoch_model_1.ckpt
beginning epoch: 2
Percent: [------------------->][419328/419350] 100% ||  LOSS: 2.074297 | POS_LOSS: 17.812391 | NEG_LOSS: 12.196445 | 
TRAINING || mean loss: 2.9094421681112204 | mean pos loss: 17.57220043986126 | mean neg loss: 13.190940938371442
beginning epoch: 3
Percent: [------------------->][419328/419350] 100% ||  LOSS: 2.470434 | POS_LOSS: 17.258129 | NEG_LOSS: 12.327210 | 
TRAINING || mean loss: 2.776450111943505 | mean pos loss: 17.001830450271658 | mean neg loss: 12.20958774736076
beginning epoch: 4
Percent: [------------------->][419328/419350] 100% ||  LOSS: 2.100240 | POS_LOSS: 16.264469 | NEG_LOSS: 11.299400 | 
TRAINING || mean loss: 2.6737185398273633 | mean pos loss: 16.92666070263319 | mean neg loss: 11.808108300188959
beginning epoch: 5
Percent: [------------------->][419328/419350] 100% ||  LOSS: 1.620027 | POS_LOSS: 16.162506 | NEG_LOSS: 9.592369 |  
TRAINING || mean loss: 2.573572034

KeyboardInterrupt: 

In [57]:
# args
EVAL_BATCH_SIZE = 200;

In [58]:
# load data
df_test = pd.read_csv("eval1_unlabelled.tsv", sep= '\t', header=None)
print('Eval Data Loaded.')

Eval Data Loaded.


In [59]:
# get scores
outputs = [];
with tf.Session(config=tf_sess_config) as sess:
  saver = tf.train.Saver();
  saver.restore(sess, r'./'+model_save_folder+'/epoch_model_10.ckpt');
  #saver.restore(sess, r'./'+model_save_folder+'/batch_model.ckpt');
  for i, row in df_test.iterrows():
    if i==0:
      batch_queries = [];
      batch_passages = [];
    if i!=0 and i%EVAL_BATCH_SIZE==0:    
      #glove_arr, glove_bmasks, elmo_arr, elmo_bmasks, n_tokens = make_glove_and_elmo_batch_data(test_batch)
      #glove_arr, glove_bmasks, n_tokens = make_glove_batch_data(test_batch) 
      aa1, _, aa3 = make_glove_batch_data(batch_queries,infer_max_len=True)
      bb1, _, bb3 = make_glove_batch_data(batch_passages,infer_max_len=True)
      result = sess.run([biLSTM_Attention.p_score],
                        feed_dict={biLSTM_Attention.keep_prob: 1.0,
                                   biLSTM_Attention.query_glove:aa1,
                                   biLSTM_Attention.pos_ans_glove:bb1,
                                   biLSTM_Attention.query_glove_seq_lengths:aa3,
                                   biLSTM_Attention.pos_ans_glove_seq_lengths:bb3})
      outputs.append(result[0].tolist());
      progressBar(i,df_test.shape[0]);
      batch_queries = [];
      batch_passages = [];
    query, passage = row[1], row[2];
    batch_queries.append(query);
    batch_passages.append(passage);
  if batch_queries:
    #glove_arr, glove_bmasks, elmo_arr, elmo_bmasks, n_tokens = make_glove_and_elmo_batch_data(test_batch)
    #glove_arr, glove_bmasks, n_tokens = make_glove_batch_data(test_batch) 
    aa1, _, aa3 = make_glove_batch_data(batch_queries,infer_max_len=True)
    bb1, _, bb3 = make_glove_batch_data(batch_passages,infer_max_len=True)
    result = sess.run([biLSTM_Attention.p_score],
                      feed_dict={biLSTM_Attention.keep_prob: 1.0,
                                 biLSTM_Attention.query_glove:aa1,
                                 biLSTM_Attention.pos_ans_glove:bb1,
                                 biLSTM_Attention.query_glove_seq_lengths:aa3,
                                 biLSTM_Attention.pos_ans_glove_seq_lengths:bb3})
    outputs.append(result[0].tolist());
    progressBar(i,df_test.shape[0]);
    batch_queries = [];
    batch_passages = [];

INFO:tensorflow:Restoring parameters from ./biLSTM_Attention/epoch_model_10.ckpt
Percent: [------------------->][104169/104170] 100% ||  : -1.000000 | 

In [60]:
outputs_ = np.hstack(outputs)
df_test['pred'] = outputs_;

In [47]:
#lol = np.hstack(outputs)
#len(lol)

In [61]:
# re-using code from Dileep
outfilename = 'biLSTM_Attention_epoch_10.tsv'
with open(outfilename,"w",encoding="utf-8") as fw:
  import math
  linelist = []
  tempscores = []
  for idx, row in tqdm(df_test.iterrows()):
      tempscores.append(row['pred'])
      #tempscores.append(str(row['pred']))
      if((idx +1)%10==0):
          tempscores-=np.min(tempscores);
          tempscores = [math.exp(s) for s in tempscores];
          expsum = sum(tempscores)
          tempscores = [str(s/expsum) for s in tempscores]
          scoreString = "\t".join(tempscores)
          qid = str(row[0])
          fw.write(qid+"\t"+scoreString+"\n")
          tempscores=[]
      #if(idx%10000==0):
      #    print(idx)

104170it [00:06, 15880.66it/s]
